In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from sympy import (symbols, simplify)
from sympy.physics.mechanics import dynamicsymbols, init_vprinting
from sympy.physics.mechanics import Lagrangian, ReferenceFrame, Point, Particle,inertia, RigidBody, angular_momentum

In [ ]:
from optibot.symbolic import lagrange, diff_to_symb, SimpLagrangesMethod
from optibot.numpy import unpack

In [ ]:
init_vprinting()

### Symbolic Problem Modelling

In [ ]:
m0, m1, m2, m3, m4, l0, l1, l2, l3, l4, t, g = symbols('m_0:5 l_0:5 t g')
I0, I1, I2, I3, I4, d0, d1, d2, d3, d4 = symbols('I_0:5 d_0:5')
q0, q1, q2, q3, q4 = dynamicsymbols('q_0:5')
m0, m1, m2, m3, m4, l0, l1, l2, l3, l4, t, g, I0, I1, I2, I3, I4, d0, d1, d2, d3, d4, q0, q1, q2, q3, q4

In [ ]:
N_in = ReferenceFrame('N')
P0 = Point('P0')
P0.set_vel(N_in, 0)

N0 = N_in.orientnew('N0', 'Axis', [q0, N_in.z])
P1 = P0.locatenew('P1', l0 * N0.y)
P1.set_vel(N_in, P1.pos_from(P0).dt(N_in))
CM0 = P0.locatenew('CM0', (l0-d0) * N0.y)
CM0.set_vel(N_in, CM0.pos_from(P0).dt(N_in))
I_0 = inertia(N0, 0, 0, I0)
body0 = RigidBody('Stance_Tibia', CM0, N0, m0, (I_0,CM0))
body0.potential_energy = m0 * g * CM0.pos_from(P0).dot(N_in.y)


N1 = N_in.orientnew('N1', 'Axis', [q1, N_in.z])
P2 = P1.locatenew('P2', l1 * N1.y)
P2.set_vel(N_in, P2.pos_from(P0).dt(N_in))
CM1 = P1.locatenew('CM1', (l1-d1) * N1.y)
CM1.set_vel(N_in, CM1.pos_from(P0).dt(N_in))
I_1 = inertia(N1, 0, 0, I1)
body1 = RigidBody('Stance_Femur', CM1, N1, m1, (I_1,CM1))
body1.potential_energy = m1 * g * CM1.pos_from(P0).dot(N_in.y)


N2 = N_in.orientnew('N2', 'Axis', [q2, N_in.z])
P3 = P2.locatenew('P3', l2 * N2.y)
P3.set_vel(N_in, P3.pos_from(P0).dt(N_in))
CM2 = P2.locatenew('CM2', d2 * N2.y)
CM2.set_vel(N_in, CM2.pos_from(P0).dt(N_in))
I_2 = inertia(N2, 0, 0, I2)
body2 = RigidBody('Torso', CM2, N2, m2, (I_2,CM2))
body2.potential_energy = m2 * g * CM2.pos_from(P0).dot(N_in.y)


N3 = N_in.orientnew('N3', 'Axis', [q3, N_in.z])
P4 = P2.locatenew('P4', -l3 * N3.y)
P4.set_vel(N_in, P4.pos_from(P0).dt(N_in))
CM3 = P2.locatenew('CM3', -d3 * N3.y)
CM3.set_vel(N_in, CM3.pos_from(P0).dt(N_in))
I_3 = inertia(N3, 0, 0, I3)
body3 = RigidBody('Swing_Femur', CM3, N3, m3, (I_3,CM3))
body3.potential_energy = m3 * g * CM3.pos_from(P0).dot(N_in.y)


N4 = N_in.orientnew('N4', 'Axis', [q4, N_in.z])
P5 = P4.locatenew('P5', -l4 * N4.y)
P5.set_vel(N_in, P5.pos_from(P0).dt(N_in))
CM4 = P4.locatenew('CM4', -d4 * N4.y)
CM4.set_vel(N_in, CM4.pos_from(P0).dt(N_in))
I_4 = inertia(N4, 0, 0, I4)
body4 = RigidBody('Swing_Tibia', CM4, N4, m4, (I_4,CM4))
body4.potential_energy = m4 * g * CM4.pos_from(P0).dot(N_in.y)

In [ ]:
Lag_simp = Lagrangian(N_in, body0, body1, body2, body3, body4)
Lag_simp

In [ ]:
from optibot.symbolic import ImplicitLagrangesMethod

In [ ]:
u0, u1, u2, u3, u4 = symbols('u_:5')
FL = [
    (N0, (u0-u1) * N_in.z),
    (N1, (u1-u2) * N_in.z),
    (N2, (u2-u3) * N_in.z),
    (N3, (u3-u4) * N_in.z),
    (N4, u4 * N_in.z)
]
LM_small = ImplicitLagrangesMethod(Lag_simp, [q0, q1, q2, q3, q4], forcelist=FL, frame=N_in)

In [ ]:
LM_small.form_lagranges_equations()

In [ ]:
impl_x = LM_small.implicit_dynamics_x

In [ ]:
impl_q = LM_small.implicit_dynamics_q

In [ ]:
from optibot.casadi import implicit_dynamic_q_to_casadi_function, implicit_dynamic_x_to_casadi_function
from optibot.symbolic import find_arguments

In [ ]:
find_arguments(impl_q, list(LM_small._q), separate_as=True, separate_lambdas=True, verbose=True)

In [ ]:
imp_dyn_x_f_cas = implicit_dynamic_x_to_casadi_function(impl_x, list(dynamicsymbols('x_0:10')), verbose=True)

In [ ]:
imp_dyn_q_f_cas = implicit_dynamic_q_to_casadi_function(impl_q, list(LM_small.q), verbose=True)

Lag_num = Lag_simp.subs([
    [m0,3.2],
    [m1,6.8],
    [m2,20],
    [m3,6.8],
    [m4,3.2],
    [l0,0.4],
    [l1,0.4],
    [l2,0.625],
    [l3,0.4],
    [l4,0.4],
    [I0,0.93],
    [I1,1.08],
    [I2,2.22],
    [I3,1.08],
    [I4,0.93],
    [d0,1.128],
    [d1,0.163],
    [d2,0.2],
    [d3,0.163],
    [d4,0.128],
    [g, 9.81]
])
Lag_num

In [ ]:
imp_dyn_q_f_cas

In [ ]:
imp_dyn_x_f_cas

In [ ]:
from optibot.symbolic import diff_to_symb_expr

In [ ]:
feet_x = P5.pos_from(P0).dot(N_in.x)
feet_x = diff_to_symb_expr(feet_x)
feet_x

In [ ]:
feet_y = P5.pos_from(P0).dot(N_in.y)
feet_y = diff_to_symb_expr(feet_y)
feet_y

In [ ]:
feet_y_vel = P5.vel(N_in).dot(N_in.y)   #pos_from(P0).dot(N_in.y)
feet_y_vel = diff_to_symb_expr(feet_y_vel)
feet_y_vel

In [ ]:
cm_pos = m0*CM0.pos_from(P0)
cm_pos += m1*CM1.pos_from(P0)
cm_pos += m2*CM2.pos_from(P0)
cm_pos += m3*CM3.pos_from(P0)
cm_pos += m4*CM4.pos_from(P0)
cm_pos = cm_pos/(m0+m1+m2+m3+m4)

sys_CM = P0.locatenew('Sys_CM', cm_pos)
sys_CM_x = simplify(sys_CM.pos_from(P0).dot(N_in.x))
sys_CM_y = simplify(sys_CM.pos_from(P0).dot(N_in.y))

In [ ]:
from sympy import lambdify

In [ ]:
import casadi as cas
from optibot.casadi import sympy2casadi

In [ ]:
sym_x = dynamicsymbols('q_0:5')
sym_x = sym_x + [ii.diff() for ii in sym_x]
sym_x = [diff_to_symb(ii) for ii in sym_x]
sym_params = list(symbols('I_0:5 d_0:5 g l_0:2 l_3 m_0:5'))
sym_add_params = [symbols('l_4'),]
sym_vars = sym_x + sym_params + sym_add_params

print(len(sym_vars), sym_vars)

In [ ]:
cas_x_args = cas.MX.sym("x", len(sym_x))
cas_params = cas.MX.sym("p", len(sym_params))
cas_add_params = cas.MX.sym("p_add", len(sym_add_params))
cas_all_vars = [cas_x_args[ii] for ii in range(len(sym_x))]
cas_all_vars += [cas_params[ii] for ii in range(len(sym_params))]
cas_all_vars += [cas_add_params[ii] for ii in range(len(sym_add_params))]
print(len(cas_all_vars), cas_all_vars)

In [ ]:
_cas_expr_temp_x = sympy2casadi(feet_x, sym_vars, cas_all_vars)
feet_x_cas = cas.Function(
        "Feet_x",
        [cas_x_args, cas_params, cas_add_params],
        [_cas_expr_temp_x,],
        ["x", "params", "additional_params"],
        ["feet_x_position"],
    )

In [ ]:
feet_x_cas

In [ ]:
_cas_expr_temp_y = sympy2casadi(feet_y, sym_vars, cas_all_vars)
feet_y_cas = cas.Function(
        "Feet_y",
        [cas_x_args, cas_params, cas_add_params],
        [_cas_expr_temp_y,],
        ["x", "params", "additional_params"],
        ["feet_y_position"],
    )

In [ ]:
feet_y_cas

In [ ]:
_cas_expr_temp_y_vel = sympy2casadi(feet_y_vel, sym_vars, cas_all_vars)
feet_y_vel_cas = cas.Function(
        "Feet_y_vel",
        [cas_x_args, cas_params, cas_add_params],
        [_cas_expr_temp_y_vel,],
        ["x", "params", "additional_params"],
        ["feet_y_speed"],
    )

In [ ]:
feet_y_vel_cas

In [ ]:
def simetric_cond_casadi(n = 5):
    x1 = cas.MX.sym('x_1', 2*n)
    x2 = cas.MX.sym('x_2', 2*n)
    cond = [x1[ii] - x2[n-1-ii] for ii in range(n)]
    cas_funcs = cas.horzcat(*cond)
    return cas.Function(
        "Sim_cond",
        [x1, x2],
        [cas_funcs,],
        ["x_1", "x2"],
        ["residue"],
    )

In [ ]:
simetric_5_links = simetric_cond_casadi(5)
simetric_5_links

In [ ]:
bodies = [body0, body1, body2, body3, body4]
points_right = [P0, P1, P2, P2, P4]
points_left = [P5, P4, P2, P2, P1]
subs_key = list(zip(dynamicsymbols('q_0:5'),dynamicsymbols('q_p_0:5')))

impact_eqs = []
for ii in range(5):
    print('calculating eq', ii)
    print('\tleft side')
    left_side = angular_momentum(points_left[ii], N_in, *bodies[:5-ii]).dot(N_in.z)
    left_side = simplify(left_side)
    print('\tright side')
    right_side = angular_momentum(points_right[ii], N_in, *bodies[ii:]).dot(N_in.z)
    right_side = simplify(right_side).subs(subs_key)
    impact_eqs.append(left_side-right_side)
#impact_eqs

In [ ]:
for ii in range(5):
    print(points_left[ii],bodies[:5-ii])
for ii in range(5):
    print(points_right[ii],bodies[ii:])

In [ ]:
def impact_cond_casadi(eqs, x1_sym, x2_sym, sym_params, sym_add_params):
    x1_sym = [diff_to_symb(ii) for ii in x1_sym]
    x2_sym = [diff_to_symb(ii) for ii in x2_sym]
    eqs = [diff_to_symb_expr(ii) for ii in eqs]

    all_vars = x1_sym + x2_sym + sym_params + sym_add_params
    n = len(x1_sym)
    cas_x1 = cas.MX.sym('x_1', n)
    cas_x2 = cas.MX.sym('x_2', n)
    cas_params = cas.MX.sym("p", len(sym_params))
    cas_add_params = cas.MX.sym("p_add", len(sym_add_params))
    cas_all_vars = [cas_x1[ii] for ii in range(n)]
    cas_all_vars += [cas_x2[ii] for ii in range(n)]
    cas_all_vars += [cas_params[ii] for ii in range(len(sym_params))]
    cas_all_vars += [cas_add_params[ii] for ii in range(len(sym_add_params))]
    
    cas_funcs = []
    for function in eqs:
        cas_funcs.append(sympy2casadi(function, all_vars, cas_all_vars))
    cas_funcs = cas.horzcat(*cas_funcs)
    return cas.Function(
        "Sim_cond",
        [cas_x1, cas_x2, cas_params, cas_add_params],
        [cas_funcs,],
        ["x_1", "x2", 'params', 'additional_params'],
        ["residue"],
    )

In [ ]:
sym_x = dynamicsymbols('q_0:5')
sym_x = sym_x + [ii.diff() for ii in sym_x]
subs_key = list(zip(dynamicsymbols('q_0:5'),dynamicsymbols('q_p_0:5')))
sym_x_2 = [ii.subs(subs_key) for ii in sym_x]
impact_cond_cas_f = impact_cond_casadi(impact_eqs, sym_x,  sym_x_2, sym_params, sym_add_params)

In [ ]:
impact_cond_cas_f

In [ ]:
sys_CM_x

In [ ]:
sym_x, sym_params

In [ ]:
sys_cm_np = lambdify([sym_x, sym_params], [sys_CM_x, sys_CM_y],'numpy')

In [ ]:
ang_mom_p0 = angular_momentum(P0, N_in, *bodies).dot(N_in.z)
ang_mom_p0_np = lambdify([sym_x, sym_params], ang_mom_p0,'numpy')

In [ ]:
ang_mom_p5 = angular_momentum(P5, N_in, *bodies).dot(N_in.z)
ang_mom_p5_np = lambdify([sym_x, sym_params, sym_add_params], ang_mom_p5,'numpy')

In [ ]:
P5_static = P5.locatenew('P5_static', 0 * N_in.y)
P5_static.set_vel(N_in, 0 * N_in.y)

In [ ]:
ang_mom_p5_static = angular_momentum(P5_static, N_in, *bodies).dot(N_in.z)
ang_mom_p5_static_np = lambdify([sym_x, sym_params, sym_add_params], ang_mom_p5_static,'numpy')

In [ ]:
angular_momentum(P5_static, N_in, bodies[-1]).dot(N_in.z)

In [ ]:
angular_momentum(P5, N_in, bodies[-1]).dot(N_in.z)

In [ ]:
from sympy.physics.mechanics import kinetic_energy, potential_energy

In [ ]:
angular_momentum(P0, N_in, bodies[0]).dot(N_in.z)

In [ ]:
system_energy = potential_energy(*bodies) + kinetic_energy(N_in, *bodies)

In [ ]:
system_energy_np = lambdify([sym_x, sym_params], system_energy,'numpy')

In [ ]:
mass_matrix_np = lambdify([sym_x, sym_params], LM_small.mass_matrix,'numpy')

In [ ]:
sym_u = symbols('u_:5')
#F_impl_np = lambdify([sym_x, sym_u, symbols('placeholder'), sym_params], LM_small.forcing,'numpy')
F_impl_np = lambdify([sym_x, sym_u, sym_params], LM_small.forcing,'numpy')

### Casadi optimization

In [ ]:
from optibot.casadi import accelrestriction2casadi
from optibot.schemes import (euler_accel_restr, trapz_accel_restr, trapz_mod_accel_restr,
                             hs_mod_accel_restr, hs_accel_restr, hs_half_x)

In [ ]:
I_0_n, I_1_n, I_2_n, I_3_n, I_4_n = 0.93, 1.08, 2.22, 1.08, 0.93
d_0_n, d_1_n, d_2_n, d_3_n, d_4_n = 0.128, 0.163, 0.2, 0.163, 0.128
g_n = 9.81
l_0_n, l_1_n, l_2_n, l_3_n, l_4_n = 0.4, 0.4, 0.625, 0.4, 0.4
m_0_n, m_1_n, m_2_n, m_3_n, m_4_n = 3.2, 6.8, 20, 6.8, 3.2
params = [
    I_0_n, I_1_n, I_2_n, I_3_n, I_4_n,
    d_0_n, d_1_n, d_2_n, d_3_n, d_4_n,
    g_n,
    l_0_n, l_1_n, l_3_n,
    m_0_n, m_1_n, m_2_n, m_3_n, m_4_n
]
additional_params = [l_4_n,]

In [ ]:
opti = cas.Opti()
p_opts = {}#{"expand":True,'ipopt.print_level':0, 'print_time':0}
s_opts = {}#{"max_iter": 10000, 'tol': 1e-26}#, 'linear_solver' : "MA27"}
opti.solver("ipopt",p_opts,
                    s_opts)

In [ ]:
N = 50
X = opti.variable(N+1,10)
X_dot = opti.variable(N+1,10)
U = opti.variable(N+1,5)
U_c = opti.variable(N,5)
X_c = opti.variable(N,10)
X_dot_c = opti.variable(N,10)

In [ ]:
T = opti.parameter()
u_m = opti.parameter()
Params_opti = opti.parameter(len(params))
Add_params_opti = opti.parameter(len(additional_params))
D = opti.parameter()

In [ ]:
#cost = cas.sum2((cas.sum1(U[:,:]**2)+cas.sum1(U[1:-1,:]**2))/N)
cost = cas.sum2((4*cas.sum1(U_c[:,:]**2) + cas.sum1(U[:,:]**2)+cas.sum1(U[1:-1,:]**2))/(3*N))
#cost = cas.sum2(cas.sum1(U**2))
opti.minimize(cost)

In [ ]:
impact_cond_cas_f

In [ ]:
#Periodic gait constraint:
opti.subject_to(simetric_5_links(X[0,:], X[-1,:]) == 0)
opti.subject_to(impact_cond_cas_f(X[-1,:], X[0,:], Params_opti, Add_params_opti) == 0)

In [ ]:
#Step size constraint:
opti.subject_to(feet_x_cas(X[-1,:], Params_opti, Add_params_opti) == D)
opti.subject_to(feet_y_cas(X[-1,:], Params_opti, Add_params_opti) == 0)

In [ ]:
#Small Feet Conditions:
opti.subject_to(U[:,0] == 0)
opti.subject_to(U_c[:,0] == 0)
opti.subject_to(feet_y_vel_cas(X[0,:], Params_opti, Add_params_opti)>0)
opti.subject_to(feet_y_vel_cas(X[-1,:], Params_opti, Add_params_opti)<0)

In [ ]:
#Feet over ground Restrictions:
for ii in range(1,N):
    opti.subject_to(feet_y_cas(X[ii,:], Params_opti, Add_params_opti) > 0)

In [ ]:
imp_dyn_x_f_cas

In [ ]:
#Dynamics Constraints:
for ii in range(N+1):
    opti.subject_to(imp_dyn_x_f_cas(X[ii,:], X_dot[ii,:], U[ii,:], [], Params_opti) == 0)
for ii in range(N):
    opti.subject_to(X_c[ii,:] == hs_half_x(X[ii,:], X[ii+1,:], X_dot[ii,:], X_dot[ii+1,:], T/N))
    opti.subject_to(imp_dyn_x_f_cas(X_c[ii,:], X_dot_c[ii,:], U_c[ii,:], [], Params_opti) == 0)

In [ ]:
#Scheme Constraints
#cas_accel_restr = accelrestriction2casadi(trapz_accel_restr, 5)
cas_accel_restr = accelrestriction2casadi(hs_accel_restr, 5, 5)
for ii in range(N):
    opti.subject_to(cas_accel_restr(X[ii,:], X[ii+1,:], X_dot[ii, 5:], X_dot[ii+1, 5:],T/N, X_dot_c[ii,5:]) == 0)

In [ ]:
opti.set_value(T, 0.7)#0.7
opti.set_value(D, 0.5)

In [ ]:
opti.set_value(Params_opti, params)
opti.set_value(Add_params_opti, additional_params)

#set initial pose fixed, different problem
opti.subject_to(X[0,:].T == [-0.3, 0.7, 0, -0.5, -0.6, 0,0,0,0,0])
for ii in range(N):
    for jj in range(5):
        opti.subject_to(U[ii,jj] < 2)
        
        opti.subject_to(U[ii,jj] >-2)

In [ ]:
q_0_guess = np.array([-0.3, 0.7, 0, -0.5, -0.6])
q_1_guess = q_0_guess[::-1]
s_arr = np.linspace(0, 1, N+1)
q_guess = np.expand_dims(q_0_guess,0)+ np.expand_dims(s_arr,1)*np.expand_dims((q_1_guess - q_0_guess),0)

q_dot_guess = (q_1_guess - q_0_guess) * np.ones([N+1,1])/opti.value(T)

In [ ]:
opti.set_initial(X[:,:5], q_guess)
opti.set_initial(X[:,5:], q_dot_guess)

opti.set_initial(X_c[:,:5], (q_guess[:-1,:]+q_guess[1:,:])/2)
opti.set_initial(X_c[:,5:], q_dot_guess[:-1,:])

opti.set_initial(X_dot[:,:5], q_dot_guess)
opti.set_initial(X_dot[:,5:], 0)

opti.set_initial(X_dot_c[:,:5], q_dot_guess[:-1,:])
opti.set_initial(X_dot_c[:,5:], 0)

opti.set_initial(U, 0)
opti.set_initial(U_c, 0)

In [ ]:
sol = opti.solve()

In [ ]:
U_sol = sol.value(U)
U_c_sol = sol.value(U_c)
X_sol = sol.value(X)
X_c_sol = sol.value(X_c)
X_dot_sol = sol.value(X_dot)
X_dot_c_sol = sol.value(X_dot_c)
T_sol = sol.value(T)
T_sol_arr = np.linspace(0, T_sol, N+1)
T_c_arr = (T_sol_arr[:-1]+T_sol_arr[1:])/2

In [ ]:
plt.figure(figsize=[14,10])
labels= ['stance anckle', 'stance knee', 'stance hip', 'swing hip', 'swing knee']
for ii in range(5):
    plt.plot(T_sol_arr,U_sol[:,ii], marker = 'o', label = labels[ii])
    plt.plot(T_c_arr,U_c_sol[:,ii], 'o', label = labels[ii] + ' u_c')
plt.grid()
plt.legend()

plt.figure(figsize=[14,10])
labels= ['stance tibia', 'stance femur', 'torso', 'swing femur', 'swing tibia']
for ii in range(5):
    plt.plot(T_sol_arr, X_sol[:,ii], marker = 'o', label = labels[ii])
    plt.plot(T_c_arr,X_c_sol[:,ii], 'o', label = labels[ii] + ' x_c')
plt.grid()
plt.legend()

In [ ]:
def chain_to_draw(x,params):
    [
    I_0_n, I_1_n, I_2_n, I_3_n, I_4_n,
    d_0_n, d_1_n, d_2_n, d_3_n, d_4_n,
    g_n,
    l_0_n, l_1_n, l_3_n,
    m_0_n, m_1_n, m_2_n, m_3_n, m_4_n
    ] = params
    points_x = [0, ]
    points_y = [0, ]
    points_x.append(points_x[-1] - l_0_n*np.sin(x[0]))
    points_x.append(points_x[-1] - l_1_n*np.sin(x[1]))
    points_x.append(points_x[-1] - l_2_n*np.sin(x[2]))
    points_x.append(points_x[-2])
    points_x.append(points_x[-1] + l_3_n*np.sin(x[3]))
    points_x.append(points_x[-1] + l_4_n*np.sin(x[4]))
    
    
    points_y.append(points_y[-1] + l_0_n*np.cos(x[0]))
    points_y.append(points_y[-1] + l_1_n*np.cos(x[1]))
    points_y.append(points_y[-1] + l_2_n*np.cos(x[2]))
    points_y.append(points_y[-2])
    points_y.append(points_y[-1] - l_3_n*np.cos(x[3]))
    points_y.append(points_y[-1] - l_4_n*np.cos(x[4]))
    
    return points_x, points_y

In [ ]:
points_x, points_y = chain_to_draw(X_sol[0], params)

In [ ]:
plt.figure(figsize=[15,15])
plt.grid()
for ii in range(0, 1, 1):
    points_x, points_y = chain_to_draw(X_sol[ii], params)
    plt.plot(points_x, points_y, marker = 'o', lw=4, ms=12, color = plt.cm.viridis(ii/N))
    

plt.gca().set_aspect('equal')

In [ ]:
total_mass = m_0_n + m_1_n + m_2_n + m_3_n + m_4_n
ang_mom_arr = [ang_mom_p0_np(X_sol[ii,:],params) for ii in range(N+1)]
ang_mom_swing_foot_arr = [ang_mom_p5_np(X_sol[ii,:],params, additional_params) for ii in range(N+1)]
ang_mom_swing_foot_static_arr = [ang_mom_p5_static_np(X_sol[ii,:],params, additional_params) for ii in range(N+1)]
cm_torque_arr = [total_mass * -g_n * sys_cm_np(X_sol[ii,:], params)[0] for ii in range(N+1)]
ang_mom_arr_deriv = np.gradient(ang_mom_arr, T_sol_arr)

plt.figure(figsize=[15,10])

plt.plot(T_sol_arr, ang_mom_arr, label = 'angular moment around stance foot')
plt.plot(T_sol_arr, ang_mom_swing_foot_arr, label = 'angular moment around swing foot')
plt.plot(T_sol_arr, ang_mom_swing_foot_static_arr, label = 'angular moment around static point at swing foot')
plt.plot(T_sol_arr, cm_torque_arr)
plt.plot(T_sol_arr, ang_mom_arr_deriv)
plt.grid()
plt.legend()

In [ ]:
system_energy_arr = [system_energy_np(X_sol[ii,:],params) for ii in range(N+1)]
system_energy_arr_deriv = np.gradient(system_energy_arr, T_sol_arr)
joint_angles = -X_sol[:,:4]+X_sol[:,1:5]
joint_speeds = -X_sol[:,5:9]+X_sol[:,6:10]
external_power = np.sum(joint_speeds*U_sol[:,1:], axis=1)

plt.figure(figsize=[15,10])

plt.plot(T_sol_arr, system_energy_arr)
plt.plot(T_sol_arr, system_energy_arr_deriv)
plt.plot(T_sol_arr, external_power)
plt.grid()

x_guess = X_sol.copy()
x_guess[:,:5] = q_guess
x_guess[:,5:] = q_dot_guess

plt.figure(figsize=[15,15])
plt.grid()
for ii in range(N+1):
    points_x, points_y = chain_to_draw(x_guess[ii], params)
    plt.plot(points_x, points_y, color = plt.cm.viridis(ii/26))
    

plt.gca().set_aspect('equal')

In [ ]:
from optibot.schemes import interpolated_array, interpolated_array_derivative
from optibot.analysis import dynamic_error_implicit

n_graph = 1000
t_arr = np.linspace(0,2,n_graph)
dyn_err_q_hs_lin, dyn_err_v_hs_lin, dyn_err_2_a_hs_lin, dyn_err_2_b_hs_lin = dynamic_error_implicit(
    x_arr=X_sol,
    u_arr=U_sol,
    t_end=T_sol,
    params = params,
    F = F_impl_np,
    M = mass_matrix_np,
    lambda_arr = None,
    scheme = 'hs_parab', 
    u_scheme='parab',
    scheme_params={'u_c':U_c_sol,
                  'x_dot_c': X_dot_c_sol,
                  'x_c': X_c_sol},
    n_interp= n_graph)


for ii in range(5):
    plt.figure(figsize=[14,8])
    plt.plot(t_arr,-dyn_err_q_hs_lin[:,ii], '-', label = f'q {ii}')
    plt.plot(np.linspace(0,2,26), np.zeros(26), 'ok')
    plt.legend()
    plt.grid()


for ii in range(5):
    plt.figure(figsize=[14,8])
    plt.plot(t_arr,dyn_err_v_hs_lin[:,ii], '-', label = f'q {ii}')
    plt.plot(np.linspace(0,2,26), np.zeros(26), 'ok')
    plt.legend()
    plt.grid()
    plt.figure(figsize=[14,8])


for ii in range(5):
    plt.figure(figsize=[14,8])
    plt.plot(t_arr,dyn_err_2_b_hs_lin[:,ii], '-', label = f'q {ii}')
    plt.plot(np.linspace(0,2,26), np.zeros(26), 'ok')
    plt.legend()
    plt.grid()
    plt.figure(figsize=[14,8])


## Sistematic comparative

In [ ]:
def q_init(N):
    q_0_guess = np.array([-0.3, 0.7, 0, -0.5, -0.6])
    q_1_guess = q_0_guess[::-1]
    s_arr = np.linspace(0, 1, N)
    q_guess = np.expand_dims(q_0_guess,0)+ np.expand_dims(s_arr,1)*np.expand_dims((q_1_guess - q_0_guess),0)
    q_dot_guess = (q_1_guess - q_0_guess) * np.ones([N,1])/opti.value(T)
    return q_guess, q_dot_guess

In [ ]:
import time
def chrono_solve(opti, solve_repetitions):
    cput0 = time.time()
    for ii in range(solve_repetitions):
        sol = opti.solve()
    cput1 = time.time()
    cpudt = (cput1-cput0)/solve_repetitions
    return sol, cpudt

In [ ]:
from optibot.pseudospectral import get_bary_extreme_f, coll_points, base_points, matrix_D_bary
def get_cost_obj_trap_int(scheme, N):
    t_arr = [-1,] + coll_points(N, scheme) + [1,]
    t_arr = [float(ii) for ii in t_arr]
    start_p_f = get_bary_extreme_f(scheme, N, mode = 'u', point = 'start')
    end_p_f = get_bary_extreme_f(scheme, N, mode = 'u', point = 'end')
    def obj_f(coefs):
        start_p = start_p_f(coefs)
        end_p = end_p_f(coefs)
        coef_list = [start_p] + [coefs[jj] for jj in range(N)] + [end_p]
        sum_res = 0
        for jj in range(N+1):
            sum_res += (coef_list[jj]**2+coef_list[jj+1]**2)*(t_arr[jj+1]-t_arr[jj])/2            
        return sum_res
    return obj_f

def get_cost_obj_trap_int_cas(scheme, N):
    u_sym = cas.SX.sym('u',N)
    u_sympy = symbols(f'c0:{N}')
    fun = get_cost_obj_trap_int(scheme, N)
    sympy_expr = fun(u_sympy)
    cas_expr = sympy2casadi(sympy_expr, u_sympy, cas.vertsplit(u_sym))
    cas_f = cas.Function(
        "cost_func",
        [u_sym,],
        [cas_expr,]
    )
    return cas_f

In [ ]:
def LG_end_p_fun_cas(N, precission=20):
    from casadi import SX, vertsplit, Function
    from optibot.casadi import sympy2casadi
    x_cas = cas.SX.sym('x',N)
    x_sympy = symbols(f'c0:{N}')
    fun = get_bary_extreme_f('LG', N, mode = 'x', point = 'end')
    sympy_expr = fun(x_sympy)
    cas_expr = sympy2casadi(sympy_expr, x_sympy, cas.vertsplit(x_cas))
    cas_f = cas.Function(
        "dynamics_q",
        [x_cas,],
        [cas_expr,]
    )
    return cas_f

In [ ]:
def opti_setup(col_points = 7, scheme = 'LGL', precission = 20, M = 1, U_dim = 1):
    opti = cas.Opti()
    opts = {'ipopt.print_level':0, 'print_time':0}
    opti.solver('ipopt', opts)
    
    opt_dict = {
        'LGL': [col_points,],
        'D2': [col_points,],
        'LG2': [col_points+2,],
        'LGLm': [col_points+2,],
        'LG': [col_points+1,]
    }
    sch_options = opt_dict[scheme]
    N = sch_options[0]
    
    if scheme == 'LGL' or scheme == 'LG':
        x_opti = opti.variable(N, 2*M)
    else:
        x_opti = opti.variable(N, M)
    
    u_opti = opti.variable(col_points, U_dim)
    tau_arr = base_points(N, scheme, precission)
    return opti, N, x_opti, u_opti, tau_arr

def opti_start_point_constr(opti, scheme, x_opti, x_dot_opti, x_s, M=1):
    if scheme == 'LGL':
        opti.subject_to(x_opti[0,:].T == x_s)
    elif scheme == 'LG':
        opti.subject_to(x_opti[0,:].T == x_s)
    elif scheme == 'D2':
        opti.subject_to(x_opti[0,:].T == x_s[:M])
        opti.subject_to(x_dot_opti[0,:].T == x_s[M:])
    elif scheme == 'LG2':
        opti.subject_to(x_opti[0,:].T == x_s[:M])
        opti.subject_to(x_dot_opti[0,:].T == x_s[M:])
    elif scheme == 'LGLm':
        opti.subject_to(x_opti[0,:].T == x_s[:M])
        opti.subject_to(x_dot_opti[0,:].T == x_s[M:])
        
def opti_end_point_constr(opti, scheme, x_opti, x_dot_opti, N, x_f, precission = 20, M=1):
    if scheme == 'LGL':
        opti.subject_to(x_opti[-1,:].T == x_f)
    elif scheme == 'LG':
        endp_f = LG_end_p_fun_cas(N, precission)
        for ii in range(2*M):
            opti.subject_to(endp_f(x_opti[:,ii]) == x_f[ii])
    elif scheme == 'D2':
        opti.subject_to(x_opti[-1,:].T == x_f[:M])
        opti.subject_to(x_dot_opti[-1,:].T == x_f[M:])
    elif scheme == 'LG2':
        opti.subject_to(x_opti[-1,:].T == x_f[:M])
        opti.subject_to(x_dot_opti[-1,:].T == x_f[M:])
    elif scheme == 'LGLm':
        opti.subject_to(x_opti[-1,:].T == x_f[:M])
        opti.subject_to(x_dot_opti[-1,:].T == x_f[M:])
        
def opti_col_point_constr(opti, scheme, x_opti, x_dot_opti, x_dot_dot_opti, u_opti, params,
                          dynam_f_x, dynam_f_q, N, M=1):
    if scheme == 'LGL':
        for ii in range(N):
            opti.subject_to(dynam_f_x(x_opti[ii,:], x_dot_opti[ii,:], u_opti[ii,:], [], params)==0)
            #opti.subject_to(x_opti[ii,M:] == x_dot_opti[ii,:M]) 
    if scheme == 'LG':
        #opti.subject_to(x_opti[0,1] == x_dot_opti[0,0]) #convert starting into semicollocation point
        for ii in range(1, N):
            opti.subject_to(dynam_f_x(x_opti[ii,:], x_dot_opti[ii,:], u_opti[ii-1,:], [], params)==0)
            #opti.subject_to(x_opti[ii,M:] == x_dot_opti[ii,:M])
    elif scheme == 'D2':
        for ii in range(N):
            opti.subject_to(dynam_f_q(x_opti[ii,:], x_dot_opti[ii,:], x_dot_dot_opti[ii,:], u_opti[ii,:], [], params)==0)
    elif scheme == 'LG2':
        for ii in range(1, N-1):
            opti.subject_to(dynam_f_q(x_opti[ii,:], x_dot_opti[ii,:], x_dot_dot_opti[ii,:], u_opti[ii-1,:], [], params)==0)
    elif scheme == 'LGLm':
        for ii in range(1, N-1):
            opti.subject_to(dynam_f_q(x_opti[ii,:], x_dot_opti[ii,:], x_dot_dot_opti[ii,:], u_opti[ii-1,:], [], params)==0)
    

def opti_read_sol_arrays(scheme, sol, x_opti, u_opti, D_mat, t_f, t_0, M):
    xx_sol = sol.value(x_opti)
    uu_sol = sol.value(u_opti)
    D_mat = np.array(D_mat)
    if scheme == 'LGL':
        qq_sol = xx_sol[:,:M]
        vv_sol = xx_sol[:,M:]
    elif scheme == 'LG':
        qq_sol = xx_sol[:,:M]
        vv_sol = xx_sol[:,M:]
    elif scheme == 'D2':
        qq_sol = xx_sol
        vv_sol = 2/(t_f - t_0) * D_mat@qq_sol
    elif scheme == 'LG2':
        qq_sol = xx_sol
        vv_sol = 2/(t_f - t_0) * D_mat@qq_sol
    elif scheme == 'LGLm':
        qq_sol = xx_sol
        vv_sol = 2/(t_f - t_0) * D_mat@qq_sol
    else:
        raise ValueError(f'Unexpected scheme {scheme}')
    return qq_sol, vv_sol, uu_sol

def extract_concepts(scheme, x_opti, x_dot_opti, x_dot_dot_opti, M):
    if scheme == 'LGL':
        qq = x_opti[:,:M]
        vv = x_opti[:,M:]
        aa = x_dot_opti[:,M:]
    elif scheme == 'LG':
        qq = x_opti[:,:M]
        vv = x_opti[:,M:]
        aa = x_dot_opti[:,M:]
    elif scheme == 'D2':
        qq = x_opti
        vv = x_dot_opti
        aa = x_dot_dot_opti
    elif scheme == 'LG2':
        qq = x_opti
        vv = x_dot_opti
        aa = x_dot_dot_opti
    elif scheme == 'LGLm':
        qq = x_opti
        vv = x_dot_opti
        aa = x_dot_dot_opti
    else:
        raise ValueError(f'Unexpected scheme {scheme}')
    return qq, vv, aa

In [ ]:
_fff = get_bary_extreme_f('LG', 3, mode = 'x', point = 'end')
_fff

In [ ]:
def end_LG_2d(xx):
    N,M = xx.shape
    res = []
    endf = get_bary_extreme_f('LG', N, mode = 'x', point = 'end')
    for jj in range(M):
        res.append(endf(xx[:,jj]))
    return cas.horzcat(*res)
        

In [ ]:
from functools import lru_cache

In [ ]:
@lru_cache(maxsize=None)
def casadi_biped(col_points = 7, scheme = "LG", solve_repetitions = 1, t_end = 0.7, step_length = 0.5, precission = 20):
    opti = cas.Opti()
    p_opts = {"expand":True,'ipopt.print_level':0, 'print_time':0}
    s_opts = {"max_iter": 10000, 'tol': 1e-26}#, 'linear_solver' : "MA27"}
    opti.solver("ipopt",p_opts,
                        s_opts)
    
    if scheme not in ['LG', 'LGL', 'D2', 'LG2', 'LGLm']:
        raise ValueError(f'Unrecognized scheme: {scheme}')
        
    opti, N, X, U, tau_arr = opti_setup(col_points, scheme, precission, M=5, U_dim=5)

    t_0 = 0
    u_m = opti.parameter()
    Params_opti = opti.parameter(len(params))
    Add_params_opti = opti.parameter(len(additional_params))
    D = opti.parameter()
    
    
    D_mat = sympy2casadi(matrix_D_bary(N, scheme,precission), [], [])
    x_dot_opti = 2/(t_end - t_0) * D_mat@X
    x_dot_dot_opti = 2/(t_end - t_0) * D_mat@x_dot_opti
    
    qq, vv, a = extract_concepts(scheme, X, x_dot_opti, x_dot_dot_opti, M = 5)
    xx = cas.horzcat(qq,vv)
    if scheme == 'LG':
        xx_f = end_LG_2d(xx)
    else:
        xx_f = xx[-1,:]
    
    
    # Cost
    #if 'parab' in scheme:
    #    cost = cas.sum2((4*cas.sum1(U_c[:,:]**2) + cas.sum1(U[:,:]**2)+cas.sum1(U[1:-1,:]**2))/(3*N))
    #else:
    #    cost = cas.sum2((cas.sum1(U[:,:]**2)+cas.sum1(U[1:-1,:]**2))/N)
    #cost = cas.sum2(cas.sum1(U**2))
    f_u_cost = get_cost_obj_trap_int_cas(scheme, col_points)
    cost = 0
    for jj in range(5):
        cost += f_u_cost(U[:,jj])
    
    opti.minimize(cost)

    #Periodic gait constraint:
    opti.subject_to(simetric_5_links(xx[0,:], xx_f) == 0)
    opti.subject_to(impact_cond_cas_f(xx[-1,:], xx[0,:], Params_opti, Add_params_opti) == 0)

    #Step size constraint:
    opti.subject_to(feet_x_cas(xx_f, Params_opti, Add_params_opti) == D)
    opti.subject_to(feet_y_cas(xx_f, Params_opti, Add_params_opti) == 0)

    #Small Feet Conditions:
    opti.subject_to(U[:,0] == 0)
    opti.subject_to(feet_y_vel_cas(xx[0,:], Params_opti, Add_params_opti)>0)
    opti.subject_to(feet_y_vel_cas(xx_f, Params_opti, Add_params_opti)<0)
    

    #Feet over ground Restrictions:
    for ii in range(1,N):
        opti.subject_to(feet_y_cas(xx[ii,:], Params_opti, Add_params_opti) > 0)

    #Dynamics Constraints:
    opti_col_point_constr(opti, scheme, X, x_dot_opti, x_dot_dot_opti, U, Params_opti,
                          imp_dyn_x_f_cas, imp_dyn_q_f_cas, N, M=5)
    

    
    

    
    opti.set_value(D, step_length)#0.5

    opti.set_value(Params_opti, params)
    opti.set_value(Add_params_opti, additional_params)
    
    q_guess, q_dot_guess = q_init(N)
    opti.set_initial(qq, q_guess)
    if scheme not in ['D2', 'LG2', 'LGLm']:
        opti.set_initial(vv, q_dot_guess)
    opti.set_initial(U, 0)
    
    

    sol, cpudt = chrono_solve(opti, solve_repetitions)

    qq_sol, vv_sol, uu_sol = opti_read_sol_arrays(scheme, sol, X, U, D_mat, t_end, t_0, M=5)
    sol_cost = sol.value(cost)
    return {
        'q':qq_sol,
        'v':vv_sol,
        'u':uu_sol,
        'cpudt':cpudt,
        #err_count,
        'cost':sol_cost
    }
    

In [ ]:
import time

In [ ]:
schemes = ['LG','LG2']#'D2', 'LGL', 
initials = ['lin']#, 'funcs']
solve_repetitions = 1
N_arr = [3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,30,35,40,45]#,50,55]#

results = {}

for scheme in schemes:
    key = scheme
    print('Problem:', key)
    results[key] = {'N_arr':N_arr}
    for N in N_arr:
        print(f'\tN = {N} , ', time.strftime('%H:%M:%S ', time.localtime(time.time())))
        results[key][N] = casadi_biped(
            col_points = N,
            scheme = scheme,
            solve_repetitions = solve_repetitions,
            t_end = 0.7,
            step_length = 0.5)

In [ ]:
for scheme in schemes:
    key = scheme
    _c = []
    _cpudt = []
    for N in results[key]['N_arr']:
        _c.append(results[key][N]['cost'])
        _cpudt.append(results[key][N]['cpudt'])
    results[key]['cost'] = np.array(_c)
    results[key]['cpudt'] = np.array(_cpudt)

In [ ]:
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
plt.rcParams.update({'font.size': 15})
oct_fig_size = [10,6]

In [ ]:
plt.figure(figsize=oct_fig_size)
for scheme in schemes:
    key = scheme
    plt.plot(results[key]['N_arr'], results[key]['cost'][:], marker = 'o', label = scheme)
plt.yscale('log')
plt.grid()
plt.legend()

In [ ]:
#titles = ['LG','LG2']#'D2', 'LGL', 
plt.figure(figsize=oct_fig_size)
for ii in range(len(schemes)):
    key = schemes[ii]
    plt.plot(results[key]['N_arr'], results[key][f'cpudt'], marker = 'o', c = f'C{ii}',label = schemes[ii])
plt.grid()
plt.legend()
#plt.yscale('log')
plt.title('Biped optimization time')
plt.xlabel('Number of collocation points')
plt.ylabel('Time (s)')
plt.tight_layout(pad = 0.0)
plt.savefig(r'LG2pics/Biped_optimization_time_vs_interval_number.pdf', format='pdf')

In [ ]:
from optibot.pseudospectral import bary_poly
def plot_u(uu, scheme, tf, color = 'k', t0 = 0, N_interp = 300):
    N = len(uu)
    tau = coll_points(N, scheme, 30)
    b_pol = bary_poly(tau, uu)
    coll_p_t = t0 + (1 + np.array(tau))*(tf - t0)/2
    tau_int_arr = np.linspace(-1, 1, N_interp)
    t_int_arr = np.linspace(t0, tf, N_interp)
    
    plt.plot(coll_p_t, uu, color+'o', label = 'control '+scheme)
    
    plt.plot(t_int_arr, b_pol(tau_int_arr), color)
    
def plot_x(qq, scheme, tf, color = 'k', t0 = 0, N_interp = 300, label = 'q'):
    N = len(qq)
    #tau = get_taus(N, scheme)
    tau = base_points(N, scheme, 30)
    b_pol = bary_poly(tau, qq)
    coll_p_t = t0 + (1 + np.array(tau))*(tf - t0)/2
    tau_int_arr = np.linspace(-1, 1, N_interp)
    t_int_arr = np.linspace(t0, tf, N_interp)
    
    plt.plot(coll_p_t, qq, color+'o', label = label + ' ' +scheme)
    
    plt.plot(t_int_arr, b_pol(tau_int_arr), color)

In [ ]:
for ii in range(5):

    N = 11
    plt.figure(figsize=[16,10])
    for jj in range(len(schemes)):
        scheme = schemes[jj]
        key = scheme
        plot_x(results[key][N]['q'][:,ii], scheme, 0.7, color = f'C{jj}', t0 = 0, N_interp = 300)
    plt.plot(T_sol_arr,X_sol[:,ii], 'k:', label = 'HS reference')
        
    plt.grid()
    plt.legend()

In [ ]:
for ii in range(5):

    N = 11
    plt.figure(figsize=[16,10])
    for jj in range(len(schemes)):
        scheme = schemes[jj]
        key = scheme
        plot_u(results[key][N]['u'][:,ii], scheme, 0.7, color = f'C{jj}', t0 = 0, N_interp = 300)
    plt.plot(T_sol_arr,U_sol[:,ii], 'k:', label = 'HS reference')
        
    plt.grid()
    plt.legend()

plt.figure(figsize=[15,15])
plt.grid()
N = 20
for ii in  range(0, N+1, 2):
    points_x, points_y = chain_to_draw(results['LG'][N]['q'][ii], params)
    plt.plot(points_x, points_y, marker = 'o', color = plt.cm.viridis(ii/N))
    

plt.gca().set_aspect('equal')

In [ ]:
from optibot.analysis import generate_G

In [ ]:
def generate_G(M, F):
    """
    Generate a function G from M and F, so that from 

            | q''  |   |                | -1   |                 |
            |      | = |  M(x, params)  |    @ | F(x, u, params) |
            |lambda|   |                |      |                 |,
    
    we can get a function G so that:
        
         q'' = G(x, u) = (M(x)^-1 @  F(x, u)) [upperside]
            
    Parameters
    ----------
    M : Function of (x, params)
        Returns a Numerical Matrix.
    F : Function of (x, u, params)
        Returns a Numerical Vector.

    Returns
    -------
    G : Function of (x, u, params)
        Equal to q'' where the collocation constraint is enforced.

    """

    def G(x, u, params):
        dim = x.shape[-1] // 2
        mm = M(x, params)
        ff = F(x, u, params)
        res = np.linalg.solve(mm,ff)
        return res[:dim]
    return G

In [ ]:
G_nump = generate_G(F = F_impl_np,
           M = mass_matrix_np,)

In [ ]:
def G_nump_q(q,v,u,params):
    q = np.array(q)
    v = np.array(v)
    x = np.concatenate((q,v), axis = -1)
    return G_nump(x,u,params)

In [ ]:
from optibot.pseudospectral import dynamic_error_pseudospectral, try_array_f

In [ ]:
from numpy import array, piecewise, linspace, expand_dims
from optibot.pseudospectral import extend_u_array, get_hermite_x, get_pol_u, get_pol_x
from optibot.pseudospectral import extend_x_arrays
from optibot.schemes import interp_2d
def _interpolations(
    qq,
    vv,
    uu,
    scheme,
    t0,
    t1,
    u_interp="pol",
    x_interp="pol",
    g_func=lambda q, v, u, p: u,
    params=[],
    n_interp=5000,
):
    """
    Generates arrays of equispaced points with values of dynamic error.
    
    If x(t) = [q(t), v(t)], and the physics equation states that x' = F(x, u),
    which is equivalent to [q', v'] = [v , G(q, v, u)] we can define the 
    dynamic errors at a point t as:
        dyn_q_err = q'(t) - v(t)
        dyn_v_err = v'(t) - G(q(t), v(t), u(t))
        dyn_2_err = q''(t) - G(q(t), v(t), u(t))
        
    'x_interp' and 'u_interp' define the way in which we interpolate the values
    of q, v and u between the given points.

    Parameters
    ----------
    qq : Numpy Array, shape = (W, N)
        Values known of q(t)
    vv : Numpy Array, shape = (W, N)
        Values known of v(t)
    uu : Numpy Array, shape = (Y, [Z])
        Values known of x(t)
    scheme : str
        Pseudospectral cheme used in the optimization.
        Acceptable values are:
            'LG'
            'LG_inv'
            'LGR'
            'LGR_inv'
            'LGL'
            'LGLm'
            'LG2'
            'D2'
    t0 : float
        starting time of interval of analysis
    t1 : float
        ending time of interval of analysis
    u_interp :  string, optional
        Model of the interpolation that must be used. The default is "pol".
        Acceptable values are:
            "pol": corresponding polynomial interpolation
            "lin": lineal interpolation
            "smooth": 3d order spline interpolation
    x_interp : string, optional
        Model of the interpolation that must be used. The default is "pol".
        Acceptable values are:
            "pol": corresponding polynomial interpolation
            "lin": lineal interpolation
            "Hermite": Hermite's 3d order spline interpolation
    g_func : Function of (q, v, u, params)
        A function of a dynamic sistem, so that
            q'' = g(q, q', u, params)
    params : list
        Physical problem parameters to be passed to F
    n_interp : int, default 5000
        number of interpolation points

    Raises
    ------
    NameError
        When an unsupported value for scheme, x_interp or u_interp is used.

    Returns
    -------
    err_q : Numpy array, shape = (n_interp, N)
        equispaced values of dynamic error q'(t) - v(t).
    err_v : Numpy array, shape = (n_interp, N)
        equispaced values of dynamic error v'(t) - G(q(t), v(t), u(t)).
    err_2 : Numpy array, shape = (n_interp, N)
        equispaced values of dynamic error q''(t) - G(q(t), q'(t), u(t)).

    """
    from scipy.interpolate import CubicHermiteSpline as hermite
    from numpy import interp, gradient, zeros_like

    scheme_opts = ["LG", "LG_inv", "LGR", "LGR_inv", "LGL", "D2", "LG2", "LGLm"]
    if scheme not in scheme_opts:
        NameError(f"Invalid scheme.\n valid options are {scheme_opts}")

    N = len(qq)
    tau_arr = linspace(-1, 1, n_interp)

    g_func = try_array_f(g_func)

    if u_interp == "pol":
        pol_u = get_pol_u(scheme, uu)
        u_arr = pol_u(tau_arr)
    elif u_interp == "lin":
        tau_u, uu = extend_u_array(uu, scheme, N)
        if len(uu.shape) == 1:
            u_arr = interp(tau_arr, tau_u, uu)
        elif len(uu.shape) == 2:
            u_arr = interp_2d(tau_arr, tau_u, uu)
        else:
            raise ValueError(
                f"U has {len(uu.shape)} dimensions, values accepted are 1 and 2"
            )
    elif u_interp == "smooth":
        tau_u, uu = extend_u_array(uu, scheme, N)
        uu_dot = gradient(uu, tau_u)
        u_arr = hermite(tau_u, uu, uu_dot)(tau_arr)
    else:
        raise NameError(
            'Invalid interpolation method for u.\n valid options are "pol", "lin", "smooth"'
        )

    if x_interp == "pol":
        tau_x = base_points(N, scheme)
        pol_q, pol_v, pol_q_d, pol_v_d, pol_q_d_d = get_pol_x(scheme, qq, vv, t0, t1)
        q_arr = pol_q(tau_arr)
        v_arr = pol_v(tau_arr)
        q_arr_d = pol_q_d(tau_arr)
        v_arr_d = pol_v_d(tau_arr)
        q_arr_d_d = pol_q_d_d(tau_arr)
    elif x_interp == "lin":
        tau_x, qq, vv = extend_x_arrays(qq, vv, scheme)
        if len(qq.shape) == 1:
            q_arr = interp(tau_arr, tau_x, qq)
            v_arr = interp(tau_arr, tau_x, vv)
        elif len(qq.shape) == 2:
            q_arr = interp_2d(tau_arr, tau_x, qq)
            v_arr = interp_2d(tau_arr, tau_x, vv)
        else:
            raise ValueError(
                f"q has {len(qq.shape)} dimensions, values accepted are 1 and 2"
            )
        
        coll_p = t0 + (1 + array(tau_x, dtype="float64")) * (t1 - t0) / 2
        t_arr_lin = linspace(t0, t1, n_interp)
        q_arr_d = find_der_polyline(t_arr_lin, coll_p, qq)
        v_arr_d = find_der_polyline(t_arr_lin, coll_p, vv)
        q_arr_d_d = zeros_like(q_arr)
    elif x_interp == "Hermite":
        tau_x, qq, vv = extend_x_arrays(qq, vv, scheme)
        aa = g_func(qq, vv, uu, params)
        her_q, her_v, her_q_d, her_v_d, her_q_d_d = get_hermite_x(
            qq, vv, aa, tau_x, t0, t1
        )
        t_arr_lin = linspace(t0, t1, n_interp)
        q_arr = her_q(t_arr_lin)
        v_arr = her_v(t_arr_lin)
        q_arr_d = her_q_d(t_arr_lin)
        v_arr_d = her_v_d(t_arr_lin)
        q_arr_d_d = her_q_d_d(t_arr_lin)
    else:
        raise NameError(
            'Invalid interpolation method for x.\n valid options are "pol", "lin", "Hermite"'
        )
    
    return q_arr, q_arr_d, v_arr, v_arr_d, q_arr_d_d, u_arr

In [ ]:
_q_arr, _q_arr_d, _v_arr, _v_arr_d, _q_arr_d_d, _u_arr = _interpolations(
    results[key][N]['q'],
    results[key][N]['v'],
    results[key][N]['u'],
    scheme,
    t0 = 0,
    t1 = 0.7,
    u_interp="pol",
    x_interp="pol",
    g_func=G_nump_q,
    params=params,
    n_interp = 1000,
)

In [ ]:
for ii in range(5):
#ii = 1

    plt.figure(figsize=[16,10])
    plot_x(results[key][N]['q'][:,ii],scheme, 2, color = 'k', t0 = 0, N_interp = 300)
    plt.plot(np.linspace(0, 2, 1000), _q_arr[:,ii], label = 'q_arr')
    plt.grid()
    plt.legend()

for ii in range(5):
#ii = 1

    plt.figure(figsize=[16,10])
    plot_x(results[key][N]['v'][:,ii],scheme, 2, color = 'k', t0 = 0, N_interp = 300)
    plt.plot(np.linspace(0, 2, 1000), _v_arr[:,ii], label = 'v_arr')
    plt.plot(np.linspace(0, 2, 1000), _q_arr_d[:,ii], label = 'q_d_arr')
    plt.grid()
    plt.legend()

In [ ]:
for ii in range(5):

    plt.figure(figsize=[16,10])
    plt.plot(np.linspace(0, 2, 1000), _g_func(_q_arr, _v_arr, _u_arr, params)[:,ii], label = 'v_arr')
    plt.plot(np.linspace(0, 2, 1000), _q_arr_d_d[:,ii], label = 'q_d_d_arr')
    plt.grid()
    plt.legend()

In [ ]:
_d_e = dynamic_error_pseudospectral(
    results[key][N]['q'],
    results[key][N]['v'],
    results[key][N]['u'],
    scheme,
    t0 = 0,
    t1 = 0.7,
    u_interp="pol",
    x_interp="pol",
    g_func=G_nump_q,
    params=params,
    n_interp = 1000,
)

In [ ]:
for ii in range(3):
    plt.figure(figsize=[14,10])
    plt.plot(np.linspace(-1, 1, 1000), _d_e[ii])
    plt.grid()
    plt.plot(coll_points(N, scheme), np.zeros_like(coll_points(N, scheme)), 'ok')

## Trapz abs err + integr

In [ ]:
N= 10
n_interp = 1000
_d_e = dynamic_error_pseudospectral(
    results['LG'][N]['q'],
    results['LG'][N]['v'],
    results['LG'][N]['u'],
    'LG',
    t0 = 0,
    t1 = 0.7,
    u_interp="pol",
    x_interp="pol",
    g_func=G_nump_q,
    params=params,
    n_interp = n_interp,
)
_d_e_2 = dynamic_error_pseudospectral(
    results['LG2'][N]['q'],
    results['LG2'][N]['v'],
    results['LG2'][N]['u'],
    'LG2',
    t0 = 0,
    t1 = 0.7,
    u_interp="pol",
    x_interp="pol",
    g_func=G_nump_q,
    params=params,
    n_interp = n_interp,
)


titles = [
    r'First order dynamic error $\varepsilon^{[1]}$,'+f' N = {N}',
    r'$\dot{v} - g(q, v, u),$'+f' N = {N}',
    r'Second order dynamic error $\varepsilon^{[2]}$,'+f' N = {N}'
]
filenames = [
    f'first_order_dynamic_error_N_{N}',
    f'v_dot_g_diff_N_{N}',
    f'second_order_dynamic_error_N_{N}'
]
units = ['rad/s', 'rad/s^2', 'rad/s^2']


for ii in range(3):
    plt.figure(figsize=oct_fig_size)
    plt.plot(np.linspace(-1, 1, n_interp), np.sum(np.abs(_d_e[ii]), axis=1), c='C0', label = 'LG')
    plt.plot(np.linspace(-1, 1, n_interp), np.sum(np.abs(_d_e_2[ii]), axis=1),c = 'C1', label = 'LG2')
    plt.grid()
    plt.plot(
        coll_points(N, scheme),
        np.zeros_like(coll_points(N, scheme)),
        'ok')
    plt.legend()
    plt.title(titles[ii])
    plt.xlabel('Time(s)')
    plt.ylabel(f'Dynamic error $({units[ii]})$')
    plt.tight_layout(pad = 0.0)
    plt.savefig(r'LG2pics/biped_'+filenames[ii]+f'.pdf', format='pdf')

In [ ]:
n_interp = 2000
for scheme in schemes:
    key = scheme 
    print('scheme: ', scheme)
    for N in results[key]['N_arr']:
        print(f'\t N:{N} , ', time.strftime('%H:%M:%S ', time.localtime(time.time())))
        _dyn_err = dynamic_error_pseudospectral(
            results[key][N]['q'],
            results[key][N]['v'],
            results[key][N]['u'],
            scheme,
            t0 = 0,
            t1 = 0.7,
            u_interp="pol",
            x_interp="pol",
            g_func=G_nump_q,
            params=params,
            n_interp = n_interp,
        )
        results[key][N]['err_q_arr'] = _dyn_err[0]
        results[key][N]['err_v_arr'] = _dyn_err[1]
        results[key][N]['err_2_arr'] = _dyn_err[2]

In [ ]:
for scheme in schemes:
    key = scheme
    print('scheme: ', scheme)
    _lis = []
    for N in results[key]['N_arr']:
        #print('\t N:', N)
        integ_err = np.trapz(np.abs(results[key][N]['err_2_arr']), np.linspace(0,2,n_interp), axis = 0)
        results[key][N]['err_2_abs_integ'] = integ_err
        _lis.append(integ_err)
    results[key]['err_2_abs_integ_vect'] = np.array(_lis)
    results[key]['err_2_abs_integ'] = np.sum(np.array(_lis), axis = 1)

In [ ]:
for ii in range(5):
    plt.figure(figsize=[16,10])
    for scheme in schemes:
        key = scheme
        plt.plot(results[key]['N_arr'], results[key]['err_2_abs_integ_vect'][:,ii], marker = 'o', label = scheme)
    plt.yscale('log')
    plt.grid()
    plt.legend()

In [ ]:
plt.figure(figsize=oct_fig_size)
for scheme in schemes:
    key = scheme
    plt.plot(results[key]['N_arr'], results[key]['err_2_abs_integ'], marker = 'o', label = scheme)
plt.yscale('log')
plt.grid()
plt.legend()
plt.title('Biped second order dynamic error $E^{[2]}$')
plt.xlabel('Number of collocation points')
plt.ylabel(f'Dynamic error ($rad/s$)')
plt.tight_layout(pad = 0.0)
plt.savefig(r'LG2pics/biped_second_order_dynamic_error_vs_interval_number.pdf', format='pdf')

from optibot.schemes import _newpoint, _calculate_missing_arrays, interp_2d, _newpoint_der
from numpy import interp
def dynamic_error_point(
    x_arr,
    u_arr,
    t_arr,
    t,
    params,
    F,
    x_dot_arr=None,
    scheme="hs_scipy",
    u_scheme="lin",
    scheme_params={},
):
    """
    Generate arrays of equispaced points with values of dynamic error.
    
    If x(t) = [q(t), v(t)], and the physics equation states that x' = F(x, u),
    which is equivalent to [q', v'] = [v , G(q, v, u)] we can define the 
    dynamic errors at a point t as:
        dyn_q_err = q'(t) - v(t)
        dyn_v_err = v'(t) - G(q(t), v(t), u(t))
        dyn_2_err_a = q''(t) - G(q(t), v(t), u(t))
        dyn_2_err_b = q''(t) - G(q(t), q'(t), u(t))
        
    'scheme' and 'u_scheme' define the way in which we interpolate the values
    of q, v and u between the given points.
        
    It is assumed that X and U start at t = 0 and are equispaced in time
    in the interval (0, t_end).
    

    Parameters
    ----------
    x_arr : Numpy Array, shape = (W, 2N)
        Values known of x(t)
    u_arr : Numpy Array, shape = (W, [Y])
        Values known of u(t)
    t_end : float
        ending time of interval of analysis
    params : list
        Physical problem parameters to be passed to F
    F : Function of (x, u, params)
        A function of a dynamic sistem, so that
            x' = F(x, u, params)
        if x_dot_arr is None and F is not, F will be used to calculate X'
    x_dot_arr : Numpy Array, optional, shape = (W, 2N), default = None
        Known values of X'
        if x_dot_arr is None, F will be used to calculate X'
    scheme : str, optional
        Scheme to be used in the X interpolation. The default is "hs_scipy".
        Acceptable values are:
            "trapz" : trapezoidal scheme compatible interpolation (not lineal!)
            "trapz_mod": modified trapezoidal scheme compatible interpolation (not lineal!)
            "hs_scipy": 3d order polynomial that satisfies continuity in x(t) and x'(t)
            "hs": Hermite-Simpson scheme compatible interpolation
            "hs_mod": modified Hermite-Simpson scheme compatible interpolation
            "hs_parab": Hermite-Simpson scheme compatible interpolation with parabolic U
            "hs_mod_parab": modified Hermite-Simpson scheme compatible interpolation with parabolic U
    u_scheme : string, optional
        Model of the interpolation that must be used. The default is "lin".
        Acceptable values are:
            "lin": lineal interpolation
            "parab": parabolic interpolation, requires central points array
            as scheme params[0]
    scheme_params :dict, optional
        Aditional parameters of the scheme. The default is {}.
    n_interp : int, optional
        Number of interpolation points. The default is 2000.

    Returns
    -------
    dyn_err_q : Numpy array, shape = (n_interp, N)
        equispaced values of dynamic error q'(t) - v(t).
    dyn_err_v : Numpy array, shape = (n_interp, N)
        equispaced values of dynamic error v'(t) - G(q(t), v(t), u(t)).
    dyn_err_2_a : Numpy array, shape = (n_interp, N)
        equispaced values of dynamic error q''(t) - G(q(t), v(t), u(t)).
    dyn_err_2_b : Numpy array, shape = (n_interp, N)
        equispaced values of dynamic error q''(t) - G(q(t), q'(t), u(t)).

    """
    if "parab" in scheme and u_scheme == "lin":
        warnings.warn(
            "You are currently using a u-parabolic interpolation for x with a lineal interpolation of u"
        )
    if "parab" in u_scheme and "parab" not in scheme:
        warnings.warn(
            "You are currently using a parabolic interpolation for u with a non u-parabolic interpolation of x"
        )
    N = x_arr.shape[0] - 1
    dim = x_arr.shape[1] // 2
    h = (t_arr[-1]-t_arr[0]) / N
    
    x_dot_arr = _calculate_missing_arrays(
        X, U, h, params, F, x_dot_arr, scheme, u_scheme, scheme_params
    )
    if u_scheme in ["min_err", "pinv_dyn"]:
        scheme_params["X"] = X
        scheme_params["scheme"] = scheme
        scheme_params["params"] = params
        scheme_params["x_dot_arr"] = x_dot_arr
        if u_scheme == "min_err":
            if F is None:
                raise ValueError(
                    "F cannot be None when using min_err as u interpolation"
                )
            scheme_params["F"] = F
            
    
    if scheme == "hs_scipy":
        X_interp = hermite(t_array, x_arr, x_dot_arr)
        x = X_interp(t)
    else:
        x = array(
                _newpoint(x_arr, x_dot_arr, h, t, params, scheme, scheme_params)
            ).flatten()
        
    if u_scheme == "lin":
        if len(u_arr.shape) == 1:
            u = interp(t, t_array, u_arr)
        elif len(u_arr.shape) == 2:
            u = interp_2d(t, t_array, u_arr)
        else:
            raise ValueError(
                f"U has {len(u_arr.shape)} dimensions, values accepted are 1 and 2"
            )
    else:
        u = array(
            _newpoint_u(U, h, t, u_scheme, scheme_params)
        ).flatten()
        
        
    if scheme == "hs_scipy":
        X_interp = hermite(t_array, x_arr, x_dot_arr)
        X_dot_interp = X_interp.derivative()
        x_d = X_dot_interp(t)
        x_d_d = X_dot_interp.derivative()(t)
    else:
        x_d = array(
                _newpoint_der(x_arr, x_dot_arr, h, t, params, scheme, 1, scheme_params)
            ).flatten()
        x_d_d = array(
                _newpoint_der(x_arr, x_dot_arr, h, t, params, scheme, 2, scheme_params)
            ).flatten()
            
    
    f_a = F(x, u, params)[dim:]
    x_q = x.copy()
    x_q[dim:] = x_d[:dim]
    f_b = F(x_q, u, params)[dim:]
    dyn_err_q = x_d[:dim] - x[dim:]
    dyn_err_v = x_d[dim:] - f_a
    dyn_err_2_a = x_d_d[:dim] - f_a
    dyn_err_2_b = x_d_d[:dim] - f_b
    return dyn_err_q, dyn_err_v, dyn_err_2_a, dyn_err_2_b

In [ ]:
schemes = ['hs_parab', 'hs_mod_parab','trapz', 'trapz_mod']#'hs', 'hs_mod', 


n_graph = 3000
t_arr = np.linspace(0,0.7,n_graph)
for scheme in schemes:
    key = scheme
    if 'parab' in scheme:
        u_scheme = 'parab'
    else:
        u_scheme = 'lin'
    print('Problem:', key)
    N_arr = results[key]['N_arr']
    for N in N_arr:
        print(f'\tN = {N}')
        dyn_err_q, dyn_err_v, _, dyn_err_2 = dynamic_error_implicit(
            x_arr=results[key][N]['x'],
            u_arr=results[key][N]['u'],
            t_end=results[key][N]['t'],
            params = params,
            F = F_impl_np,
            M = mass_matrix_np,
            scheme = scheme, 
            u_scheme = u_scheme,
            scheme_params={'u_c':results[key][N]['u_c'],
                          'x_dot_c': results[key][N]['x_dot_c'],
                          'x_c': results[key][N]['x_c']},
            n_interp= n_graph)
        results[key][N]['dyn_err_q'] = dyn_err_q
        results[key][N]['dyn_err_v'] = dyn_err_v
        results[key][N]['dyn_err_2'] = dyn_err_2

In [ ]:
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
plt.rcParams.update({'font.size': 15})
oct_fig_size = [10,6]

In [ ]:
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

schemes = ['hs_parab','hs_mod_parab', 'trapz', 'trapz_mod']
titles = ['Hermite Simpson','2nd order Hermite Simpson', 'Trapezoidal', '2nd order Trapezoidal']
colors = ['b', 'orange', 'g', 'r', 'purple']
n_int = len(t_arr)
N = 25
interv_n = (N * t_arr)/results[scheme][N]['t']
for kk in range(len(schemes)):
    scheme = schemes[kk]
    plt.figure(figsize=[14,8])
    for ii in range(5):
        cut_p = 0
        for ll in range(1,N+1):
            jj = np.searchsorted(interv_n, ll)
            plt.plot(t_arr[cut_p:jj],results[scheme][N]['dyn_err_q'][cut_p:jj,ii], '-', c = colors[ii], label = f'$q_{ii+1}$' if cut_p == 0 else None)
            cut_p = jj
    plt.plot(np.linspace(0,results[scheme][N]['t'],N+1), np.zeros(N+1), 'ok')
    plt.legend()
    plt.grid()
    if kk == 1:
        plt.ylim([-0.00001, 0.00001])
    elif kk == 3:
        plt.ylim([-0.001, 0.001])
    plt.title(r'First order dynamic error $\varepsilon^{[1]}_{q_i}$,'+f' {titles[kk]} scheme')
    plt.xlabel('Time(s)')
    plt.ylabel('Dynamic error $(rad/s)$')
    plt.tight_layout(pad = 0.0)
    sch_type = titles[kk].replace(' ','_')
    plt.savefig(f'5_link_First_Order_Dynamic_Error_{sch_type}_scheme.eps', format='eps')


schemes = ['hs_parab','hs_mod_parab', 'trapz', 'trapz_mod']
titles = ['Hermite Simpson','2nd order Hermite Simpson', 'Trapezoidal', '2nd order Trapezoidal']
colors = ['b', 'orange', 'g', 'r', 'purple']
n_int = len(t_arr)
N = 25
interv_n = (N * t_arr)/results[scheme][N]['t']
for kk in range(len(schemes)):
    scheme = schemes[kk]
    plt.figure(figsize=[14,8])
    for ii in range(5):
        cut_p = 0
        for ll in range(1,N+1):
            jj = np.searchsorted(interv_n, ll)
            plt.plot(t_arr[cut_p:jj],results[scheme][N]['dyn_err_2'][cut_p:jj,ii], '-', c = colors[ii], label = f'$q_{ii+1}$' if cut_p == 0 else None)
            cut_p = jj
    plt.plot(results[scheme][N]['t_array'], np.zeros(N+1), 'ok', label = 'knot & collocation points')
    if 'hs' in scheme:
        plt.plot(results[scheme][N]['t_c_array'], np.zeros(N), 'ow', markeredgecolor='b', label = 'collocation points')
        plt.ylim([-0.08, 0.08])
    else:
        plt.ylim([-1.75, 1.75])
    plt.legend()
    plt.grid()
    plt.title(r'Second order dynamic error $\varepsilon^{{[2]}}_{{q_i}}$,'+f' {titles[kk]} scheme')
    plt.xlabel('Time(s)')
    plt.ylabel('Dynamic error $(rad/s^2)$')
    plt.tight_layout(pad = 0.0)
    sch_type = titles[kk].replace(' ','_')
    plt.savefig(f'5_link_Second_Order_Dynamic_Error_{sch_type}_scheme.eps', format='eps')


In [ ]:
def arr_mod(x):
    x_1 = np.sum(x*x, axis=1)
    return np.sqrt(x_1)
def arr_sum(x):
    return np.sum(np.abs(x), axis = 1)
def arr_max(x):
    return np.max(np.abs(x), axis = 1)

In [ ]:
schemes = ['hs_mod_parab','hs_parab']#, 'trapz', 'trapz_mod']
titles = ['2nd order Hermite Simpson','Hermite Simpson']#, 'Trapezoidal', 'Modified Trapezoidal']
colors = ['b', 'orange', 'g', 'r', 'purple']
funcs = [arr_sum,]#arr_mod,  arr_max
func_tittles = ['Module of', 'Sum of absolute', 'Maximum of absolute']
y_max_list = [0.12, 0.2, 0.09]
n_int = len(t_arr)
N = 25
interv_n = (N * t_arr)/results[scheme][N]['t']
for ii in range(1):
    plt.figure(figsize=oct_fig_size)
    for kk in [1,0]:
        scheme = schemes[kk]
        cut_p = 0
        for ll in range(1,N+1):
            jj = np.searchsorted(interv_n, ll)
            y_plot = funcs[ii](results[scheme][N]['dyn_err_2'])
            plt.plot(t_arr[cut_p:jj],y_plot[cut_p:jj], '-', c = f'C{kk}', label = titles[kk] if cut_p == 0 else None)
            cut_p = jj
    plt.plot(results[scheme][N]['t_array'], np.zeros(N+1), 'ok', label = 'knot & collocation points')
    plt.plot(results[scheme][N]['t_c_array'], np.zeros(N), 'ow', markeredgecolor='k', label = 'collocation points')
    plt.legend()
    plt.grid()
    plt.ylim([-0.01,y_max_list[ii]])
    plt.title(r'Second order dynamic error $\varepsilon^{[2]}$,'+f' N = {N}')
    plt.xlabel('Time(s)')
    plt.ylabel('Dynamic error $(rad/s^2)$')
    plt.tight_layout(pad = 0.0)
    sch_type = func_tittles[ii].replace(' ','_')
    plt.savefig(f'5_link_HS_N{N}_second_order_dynamic_error.pdf', format='pdf')
    


In [ ]:
schemes = ['trapz', 'trapz_mod']#, 'trapz', 'trapz_mod']
titles = ['Trapezoidal', '2nd order Trapezoidal']#, 'Trapezoidal', 'Modified Trapezoidal']
funcs = [arr_sum,]#arr_mod,  arr_max
#func_tittles = ['Module of', 'Sum of absolute', 'Maximum of absolute']
y_max_list = [0.12, 0.2, 0.09]
n_int = len(t_arr)
N = 50
interv_n = (N * t_arr)/results[scheme][N]['t']
for ii in range(1):
    plt.figure(figsize=oct_fig_size)
    for kk in range(2):
        scheme = schemes[kk]
        cut_p = 0
        for ll in range(1,N+1):
            jj = np.searchsorted(interv_n, ll)
            y_plot = funcs[ii](results[scheme][N]['dyn_err_2'])
            plt.plot(t_arr[cut_p:jj],y_plot[cut_p:jj], '-', c = f'C{kk+2}', label = titles[kk] if cut_p == 0 else None)
            cut_p = jj
    plt.plot(results[scheme][N]['t_array'], np.zeros(N+1), 'ok', label = 'knot & collocation points')
    #plt.plot(results[scheme][N]['t_c_array'], np.zeros(N), 'ow', markeredgecolor='k', label = 'collocation points')
    plt.legend()
    plt.grid()
    #plt.ylim([-0.01,y_max_list[ii]])
    plt.title(r'Second order dynamic error $\varepsilon^{[2]}$,'+f' N = {N}')
    plt.xlabel('Time(s)')
    plt.ylabel('Dynamic error $(rad/s^2)$')
    plt.tight_layout(pad = 0.0)
    sch_type = func_tittles[ii].replace(' ','_')
    plt.savefig(f'5_link_Trapezoidal_N{N}_second_order_dynamic_error.pdf', format='pdf')
    


In [ ]:
def state_error(t_arr, dyn_err, N,):
    n_int = len(t_arr)
    interv_n = (N * t_arr)//2
    err = []
    cut_p = 0
    for ii in range(1,N+1):
        jj = np.searchsorted(interv_n, ii)
        err_point = np.trapz(np.abs(dyn_err[cut_p:jj+1]), t_arr[cut_p:jj+1], axis=0)
        err.append(err_point)
        cut_p = jj
    err = np.array(err, dtype = float)
    return err

def total_state_error(t_arr, dyn_err):
    errors = np.trapz(dyn_err, t_arr, axis=0)
    return errors


def total_state_error_abs(t_arr, dyn_err):
    errors = np.trapz(np.abs(dyn_err), t_arr, axis=0)
    return errors

In [ ]:
def integ_state_error(t_arr, dyn_err):
    int_x = np.zeros_like(dyn_err)
    arr_len = dyn_err.shape[0]
    for ii in range(1,arr_len):
        int_x[ii,:] = np.trapz(dyn_err[:ii+1,:], t_arr[:ii+1], axis=0)
    return int_x

In [ ]:
schemes = ['hs_parab', 'hs_mod_parab','trapz', 'trapz_mod']#'hs', 'hs_mod', 

t_arr = np.linspace(0,0.7,n_graph)
for scheme in schemes:
    key = scheme
    print('Problem:', key)
    N_arr = results[key]['N_arr']
    for N in N_arr:
        print(f'\tN = {N}')
        for letter in 'qv2':
            results[key][N][f'integ_dyn_err_{letter}']= total_state_error(t_arr, results[scheme][N][f'dyn_err_{letter}'])
            results[key][N][f'module_dyn_err_{letter}']= np.sqrt(np.dot(results[key][N][f'integ_dyn_err_{letter}'], results[key][N][f'integ_dyn_err_{letter}']))
            results[key][N][f'sum_dyn_err_{letter}'] = np.sum(np.abs(results[key][N][f'integ_dyn_err_{letter}']))
        

In [ ]:
schemes = ['hs_parab', 'hs_mod_parab','trapz', 'trapz_mod']#'hs', 'hs_mod', 

t_arr = np.linspace(0,0.7,n_graph)
for scheme in schemes:
    key = scheme
    print('Problem:', key)
    N_arr = results[key]['N_arr']
    for N in N_arr:
        print(f'\tN = {N}')
        results[key][N]['integ_err_2'] = integ_state_error(t_arr, results[key][N]['dyn_err_2'])
        results[key][N]['twice_integ_err_2'] = integ_state_error(t_arr, results[key][N]['integ_err_2'])
        

In [ ]:
sch = [['hs_parab','hs_mod_parab'],['trapz', 'trapz_mod']]
tit = [['Hermite Simpson','2nd order Hermite Simpson'],['Trapezoidal', '2nd order Trapezoidal']]
colors = [f'C{ii}' for ii in [1,0,2,3]]
n_int = len(t_arr)
N_hh = [25,50]
t_end = 0.7
for hh in range(2):
    schemes = sch[hh]
    titles = tit[hh]
    N = N_hh[hh]
    interv_n = (N * t_arr)/2
    for ii in range(5):
        plt.figure(figsize=oct_fig_size)
        for kk in range(len(schemes)):
            scheme = schemes[kk]
            key = scheme
            cut_p = 0
            for ll in range(1,N+1):
                jj = np.searchsorted(interv_n, ll)
                plt.plot(t_arr[cut_p:jj],results[key][N]['integ_err_2'][cut_p:jj,ii], '-', c = colors[2*hh+kk], label = titles[kk] if cut_p == 0 else None)
                cut_p = jj
        plt.plot(np.linspace(0,t_end,N+1), np.zeros(N+1), 'ok', label = 'knot & collocation points')
        if hh == 0:
            plt.plot(np.linspace(0,t_end,2*N+1)[1::2], np.zeros(N), 'ow', markeredgecolor='k', label = 'collocation points')
        plt.legend()
        plt.grid()
        #plt.ylim([-0.00022, 0.00022])
        plt.title(r'Integrated Second order dynamic error $\int_{0}^{t}\varepsilon^{[2]}_{q_'+f'{ii+1}}}'+r'(\tau) d\tau$,'+f'\n{titles[0]} schemes, N = {N}')
        plt.xlabel('Time(s)')
        units = 'm/s' if ii == 0 else'rad/s'
        plt.ylabel(f'Dynamic error $({units})$')
        plt.tight_layout(pad = 0.0)
        sch_type = titles[0].replace(' ','_')
        plt.savefig(f'Biped_Integrated_Second_Order_Dynamic_Error_q_{ii+1}_{sch_type}_schemes_N_{N}.png', format='png')

In [ ]:
sch = [['hs_parab','hs_mod_parab'],['trapz', 'trapz_mod']]
tit = [['Hermite Simpson','2nd order Hermite Simpson'],['Trapezoidal', '2nd order Trapezoidal']]
colors = [f'C{ii}' for ii in [1,0,2,3]]
n_int = len(t_arr)
N_hh = [25,50]
t_end = 0.7
for hh in range(2):
    schemes = sch[hh]
    titles = tit[hh]
    N = N_hh[hh]
    interv_n = (N * t_arr)/2
    for ii in range(5):
        plt.figure(figsize=oct_fig_size)
        for kk in range(len(schemes)):
            scheme = schemes[kk]
            key = scheme
            cut_p = 0
            for ll in range(1,N+1):
                jj = np.searchsorted(interv_n, ll)
                plt.plot(t_arr[cut_p:jj],results[key][N]['twice_integ_err_2'][cut_p:jj,ii], '-', c = colors[2*hh+kk], label = titles[kk] if cut_p == 0 else None)
                cut_p = jj
        plt.plot(np.linspace(0,t_end,N+1), np.zeros(N+1), 'ok', label = 'knot & collocation points')
        if hh == 0:
            plt.plot(np.linspace(0,t_end,2*N+1)[1::2], np.zeros(N), 'ow', markeredgecolor='k', label = 'collocation points')
        plt.legend()
        plt.grid()
        #plt.ylim([-0.00022, 0.00022])
        plt.title(r'Twice integrated second order dynamic error $\int_{0}^{t}\int_{0}^{\xi}\varepsilon^{[2]}_{q_'+f'{ii+1}}}'+r'(\tau) d\tau d\xi$,'+f'\n{titles[0]} schemes, N = {N}')
        plt.xlabel('Time(s)')
        units = 'm' if ii == 0 else'rad'
        plt.ylabel(f'Dynamic error $({units})$')
        plt.tight_layout(pad = 0.0)
        sch_type = titles[0].replace(' ','_')
        plt.savefig(f'Biped_Twice_Integrated_Second_Order_Dynamic_Error_q_{ii+1}_{sch_type}_schemes_N_{N}.png', format='png')

In [ ]:
schemes = ['hs_mod_parab','hs_parab']#, 'trapz', 'trapz_mod']
titles = ['2nd order Hermite Simpson','Hermite Simpson']#, 'Trapezoidal', 'Modified Trapezoidal']
colors = ['b', 'orange', 'g', 'r', 'purple']
funcs = [arr_sum,]#arr_mod,  arr_max
func_tittles = ['Module of', 'Sum of absolute', 'Maximum of absolute']
y_max_list = [0.12, 0.2, 0.09]
n_int = len(t_arr)
N = 25
interv_n = (N * t_arr)/results[scheme][N]['t']
for ii in range(1):
    plt.figure(figsize=oct_fig_size)
    for kk in [1,0]:
        scheme = schemes[kk]
        cut_p = 0
        for ll in range(1,N+1):
            jj = np.searchsorted(interv_n, ll)
            y_plot = funcs[ii](results[scheme][N]['integ_err_2'])
            plt.plot(t_arr[cut_p:jj],y_plot[cut_p:jj], '-', c = f'C{kk}', label = titles[kk] if cut_p == 0 else None)
            cut_p = jj
    plt.plot(results[scheme][N]['t_array'], np.zeros(N+1), 'ok', label = 'knot & collocation points')
    plt.plot(results[scheme][N]['t_c_array'], np.zeros(N), 'ow', markeredgecolor='k', label = 'collocation points')
    plt.legend()
    plt.grid()
    #plt.ylim([-0.01,y_max_list[ii]])
    plt.title(r'Sum of absolute Integrated Second order dynamic error $\Sigma |\int_{0}^{t}\varepsilon^{[2]}_{q_'+f'{ii+1}}}'+r'(\tau) d\tau|$,'+f' HS schemes, N = {N}')
    plt.xlabel('Time(s)')
    plt.ylabel('Dynamic error $(rad/s)$')
    plt.tight_layout(pad = 0.0)
    sch_type = func_tittles[ii].replace(' ','_')
    plt.savefig(f'5_link_HS_N{N}_integrated_second_order_dynamic_error.png', format='png')
    


In [ ]:
schemes = ['trapz', 'trapz_mod']#
titles = ['Trapezoidal', 'Modified Trapezoidal']#
colors = ['b', 'orange', 'g', 'r', 'purple']
funcs = [arr_sum,]#arr_mod,  arr_max
func_tittles = ['Module of', 'Sum of absolute', 'Maximum of absolute']
y_max_list = [0.12, 0.2, 0.09]
n_int = len(t_arr)
N = 25
interv_n = (N * t_arr)/results[scheme][N]['t']
for ii in range(1):
    plt.figure(figsize=oct_fig_size)
    for kk in [1,0]:
        scheme = schemes[kk]
        cut_p = 0
        for ll in range(1,N+1):
            jj = np.searchsorted(interv_n, ll)
            y_plot = funcs[ii](results[scheme][N]['integ_err_2'])
            plt.plot(t_arr[cut_p:jj],y_plot[cut_p:jj], '-', c = f'C{kk+2}', label = titles[kk] if cut_p == 0 else None)
            cut_p = jj
    plt.plot(results[scheme][N]['t_array'], np.zeros(N+1), 'ok', label = 'knot & collocation points')
    plt.plot(results[scheme][N]['t_c_array'], np.zeros(N), 'ow', markeredgecolor='k', label = 'collocation points')
    plt.legend()
    plt.grid()
    #plt.ylim([-0.01,y_max_list[ii]])
    plt.title(r'Sum of absolute Integrated Second order dynamic error $\Sigma |\int_{0}^{t}\varepsilon^{[2]}_{q_'+f'{ii+1}}}'+r'(\tau) d\tau|$,'+f' Trapz schemes, N = {N}')
    plt.xlabel('Time(s)')
    plt.ylabel('Dynamic error $(rad/s)$')
    plt.tight_layout(pad = 0.0)
    sch_type = func_tittles[ii].replace(' ','_')
    plt.savefig(f'5_link_Trapz_N{N}_integrated_second_order_dynamic_error.png', format='png')
    


In [ ]:
schemes = ['hs_mod_parab','hs_parab']#, 'trapz', 'trapz_mod']
titles = ['2nd order Hermite Simpson','Hermite Simpson']#, 'Trapezoidal', 'Modified Trapezoidal']
colors = ['b', 'orange', 'g', 'r', 'purple']
funcs = [arr_sum,]#arr_mod,  arr_max
func_tittles = ['Module of', 'Sum of absolute', 'Maximum of absolute']
y_max_list = [0.12, 0.2, 0.09]
n_int = len(t_arr)
N = 25
interv_n = (N * t_arr)/results[scheme][N]['t']
for ii in range(1):
    plt.figure(figsize=oct_fig_size)
    for kk in [1,0]:
        scheme = schemes[kk]
        cut_p = 0
        for ll in range(1,N+1):
            jj = np.searchsorted(interv_n, ll)
            y_plot = funcs[ii](results[scheme][N]['twice_integ_err_2'])
            plt.plot(t_arr[cut_p:jj],y_plot[cut_p:jj], '-', c = f'C{kk}', label = titles[kk] if cut_p == 0 else None)
            cut_p = jj
    plt.plot(results[scheme][N]['t_array'], np.zeros(N+1), 'ok', label = 'knot & collocation points')
    plt.plot(results[scheme][N]['t_c_array'], np.zeros(N), 'ow', markeredgecolor='k', label = 'collocation points')
    plt.legend()
    plt.grid()
    #plt.ylim([-0.01,y_max_list[ii]])
    plt.title(r'Sum of absolute twice integrated second order dynamic error $\Sigma |\int_{0}^{t}\int_{0}^{\xi}\varepsilon^{[2]}_{q_'+f'{ii+1}}}'+r'(\tau) d\tau d\xi|$,'+f' HS schemes, N = {N}')
    plt.xlabel('Time(s)')
    plt.ylabel('Dynamic error $(rad)$')
    plt.tight_layout(pad = 0.0)
    sch_type = func_tittles[ii].replace(' ','_')
    plt.savefig(f'5_link_HS_N{N}_twice_integrated_second_order_dynamic_error.png', format='png')
    


In [ ]:
schemes = ['trapz', 'trapz_mod']#
titles = ['Trapezoidal', 'Modified Trapezoidal']#
colors = ['b', 'orange', 'g', 'r', 'purple']
funcs = [arr_sum,]#arr_mod,  arr_max
func_tittles = ['Module of', 'Sum of absolute', 'Maximum of absolute']
y_max_list = [0.12, 0.2, 0.09]
n_int = len(t_arr)
N = 25
interv_n = (N * t_arr)/results[scheme][N]['t']
for ii in range(1):
    plt.figure(figsize=oct_fig_size)
    for kk in [1,0]:
        scheme = schemes[kk]
        cut_p = 0
        for ll in range(1,N+1):
            jj = np.searchsorted(interv_n, ll)
            y_plot = funcs[ii](results[scheme][N]['twice_integ_err_2'])
            plt.plot(t_arr[cut_p:jj],y_plot[cut_p:jj], '-', c = f'C{kk+2}', label = titles[kk] if cut_p == 0 else None)
            cut_p = jj
    plt.plot(results[scheme][N]['t_array'], np.zeros(N+1), 'ok', label = 'knot & collocation points')
    plt.plot(results[scheme][N]['t_c_array'], np.zeros(N), 'ow', markeredgecolor='k', label = 'collocation points')
    plt.legend()
    plt.grid()
    #plt.ylim([-0.01,y_max_list[ii]])
    plt.title(r'Sum of absolute twice integrated second order dynamic error $\Sigma |\int_{0}^{t}\int_{0}^{\xi}\varepsilon^{[2]}_{q_'+f'{ii+1}}}'+r'(\tau) d\tau d\xi|$,'+f' Trapz schemes, N = {N}')
    plt.xlabel('Time(s)')
    plt.ylabel('Dynamic error $(rad)$')
    plt.tight_layout(pad = 0.0)
    sch_type = func_tittles[ii].replace(' ','_')
    plt.savefig(f'5_link_Trapz_N{N}_twice_integrated_second_order_dynamic_error.png', format='png')
    


In [ ]:
schemes = ['hs_parab', 'hs_mod_parab','trapz', 'trapz_mod']#'hs', 'hs_mod', 

t_arr = np.linspace(0,0.7,n_graph)
for scheme in schemes:
    key = scheme
    print('Problem:', key)
    N_arr = results[key]['N_arr']
    _sum = []
    for N in N_arr:
        results[key][N]['twice_integ_dyn_err_2'] = results[key][N]['twice_integ_err_2'][-1,:]
        results[key][N][f'sum_twice_dyn_err_2'] = np.sum(np.abs(results[key][N]['twice_integ_dyn_err_2']))
        _sum.append(results[key][N][f'sum_twice_dyn_err_2'])
    results[key][f'sum_twice_dyn_err_2_array'] = np.array(_sum)

In [ ]:
for scheme in schemes:
    key = scheme
    print('Problem:', key)
    N_arr = results[key]['N_arr']
    
    for letter in 'qv2':
        list_mod = []
        list_sum = []
        for N in N_arr:
            #print(f'\tN = {N}')
            list_mod.append(results[key][N][f'module_dyn_err_{letter}'])
            list_sum.append(results[key][N][f'sum_dyn_err_{letter}'])
        results[key][f'module_dyn_err_{letter}_array'] = np.array(list_mod)
        results[key][f'sum_dyn_err_{letter}_array'] = np.array(list_sum)

In [ ]:
for scheme in results.keys():
    if 'hs' in scheme:
        n_coll = np.array(results[scheme]['N_arr'])*2-1
        results[scheme]['N_coll_arr'] = n_coll
    else:
        results[scheme]['N_coll_arr'] = results[scheme]['N_arr']
    print(scheme, results[scheme]['N_arr'],results[scheme]['N_coll_arr'])

schemes = ['hs_mod_parab','hs_parab', 'trapz', 'trapz_mod']
titles = ['2nd order Hermite Simpson', 'Hermite Simpson','Trapezoidal', '2nd order Trapezoidal']
plt.figure(figsize=[14,8])
for ii in range(4):
    key = schemes[ii]
    plt.plot(results[key]['N_arr'], results[key][f'module_dyn_err_2_array'], marker = 'o',label = titles[ii])
plt.grid()
plt.legend()
plt.yscale('log')
plt.title('Module second order dynamic error')
plt.xlabel('Number of intervals')
plt.ylabel('Dynamic error ($rad/s^2$)')
plt.savefig(f'5_link_Module_second_order_dynamic_error_vs_interval_number.eps', format='eps')

In [ ]:
schemes_graph = ['hs_mod_parab', 'hs_parab', 'trapz', 'trapz_mod']
titles = ['2nd order Hermite Simpson', 'Hermite Simpson','Trapezoidal', '2nd order Trapezoidal']
colors = [f'C{ii}' for ii in range(9)]
data_array = ['err_q_acum','err_v_acum','err_2_b_acum','cpudt']
initial = 'lin'


data_key = data_array[2]
for qq in range(5):
    #plt.figure(figsize=[10,6])
    #plt.title(f'Biped second order dynamic error $|E^{{*[2]}}_{{q_{qq+1}}}|$')
    for ii in [2,3,1,0]:
        scheme = schemes_graph[ii]
        key = scheme #+ '_' + initial
        print(titles[ii], f'Error 2 q_{qq+1}:')
        for nn in results[key]['N_arr']:
            print('\tN:', nn, 'Error:', results[key][nn]['integ_dyn_err_2'][qq])
        N_arr = results[key]['N_arr']
    #    if len(results[key][data_key].shape) == 1:
    #        plt.plot(N_arr,np.abs(results[key][data_key]), marker = 'o', c = f'C{ii}',label = titles[ii])
    #    else:
    #        plt.plot(N_arr,np.abs(results[key][data_key][:,qq]), marker = 'o', c = f'C{ii}',label = titles[ii])
    #plt.yscale('log')
    #plt.xlabel('Number of intervals')
    #plt.grid()
    #plt.legend()
    units = 'm/s' if qq == 0 else'rad/s'
    #plt.ylabel(f'Dynamic error $({units})$')
    #plt.tight_layout(pad = 0.0)
    #plt.savefig(f'Cartpole_Integrated_Second_Order_Dynamic_Error_q_{qq+1}_vs_N.pdf', format='pdf')


In [ ]:
schemes_graph = ['hs_mod_parab', 'hs_parab', 'trapz', 'trapz_mod']
titles = ['2nd order Hermite Simpson', 'Hermite Simpson','Trapezoidal', '2nd order Trapezoidal']
colors = [f'C{ii}' for ii in range(9)]
data_array = ['err_q_acum','err_v_acum','err_2_b_acum','cpudt']
initial = 'lin'


data_key = data_array[2]
for qq in range(5):
    #plt.figure(figsize=[10,6])
    #plt.title(f'Biped second order dynamic error $|E^{{*[2]}}_{{q_{qq+1}}}|$')
    for ii in [2,3,1,0]:
        scheme = schemes_graph[ii]
        key = scheme #+ '_' + initial
        print(titles[ii], f'Error 2 q_{qq+1}:')
        for nn in results[key]['N_arr']:
            print('\tN:', nn, 'Error:', results[key][nn]['twice_integ_dyn_err_2'][qq])
        N_arr = results[key]['N_arr']
    #    if len(results[key][data_key].shape) == 1:
    #        plt.plot(N_arr,np.abs(results[key][data_key]), marker = 'o', c = f'C{ii}',label = titles[ii])
    #    else:
    #        plt.plot(N_arr,np.abs(results[key][data_key][:,qq]), marker = 'o', c = f'C{ii}',label = titles[ii])
    #plt.yscale('log')
    #plt.xlabel('Number of intervals')
    #plt.grid()
    #plt.legend()
    units = 'm/s' if qq == 0 else'rad/s'
    #plt.ylabel(f'Dynamic error $({units})$')
    #plt.tight_layout(pad = 0.0)
    #plt.savefig(f'Cartpole_Integrated_Second_Order_Dynamic_Error_q_{qq+1}_vs_N.pdf', format='pdf')


In [ ]:
schemes = ['hs_mod_parab','hs_parab', 'trapz', 'trapz_mod']
titles = ['2nd order Hermite Simpson', 'Hermite Simpson','Trapezoidal', '2nd order Trapezoidal']
plt.figure(figsize=oct_fig_size)
for ii in [2,3,1,0]:
    key = schemes[ii]
    plt.plot(results[key]['N_arr'], results[key][f'sum_dyn_err_2_array'], marker = 'o', c = f'C{ii}',label = titles[ii])
plt.grid()
plt.legend()
plt.yscale('log')
plt.title('5-link biped second order dynamic error $\Sigma |E^{*[2]}_i|$')
plt.xlabel('Number of intervals')
plt.ylabel('Dynamic error ($rad/s$)')
plt.tight_layout(pad = 0.0)
plt.savefig(f'5_link_Sum_second_order_dynamic_error_vs_interval_number.png', format='png')

In [ ]:
schemes = ['hs_mod_parab','hs_parab', 'trapz', 'trapz_mod']
titles = ['2nd order Hermite Simpson', 'Hermite Simpson','Trapezoidal', '2nd order Trapezoidal']
plt.figure(figsize=oct_fig_size)
for ii in [2,3,1,0]:
    key = schemes[ii]
    plt.plot(results[key]['N_arr'], results[key][f'sum_twice_dyn_err_2_array'], marker = 'o', c = f'C{ii}',label = titles[ii])
plt.grid()
plt.legend()
plt.yscale('log')
plt.title('5-link biped twice integrated second order dynamic error $\Sigma |EE^{*[2]}_i|$')
plt.xlabel('Number of intervals')
plt.ylabel('Dynamic error ($rad$)')
plt.tight_layout(pad = 0.0)
plt.savefig(f'5_link_Sum_twice_integ_second_order_dynamic_error_vs_interval_number.png', format='png')

schemes = ['hs_mod_parab','hs_parab', 'trapz', 'trapz_mod']
titles = ['Modified Hermite Simpson', 'Hermite Simpson','Trapezoidal', 'Modified Trapezoidal']
plt.figure(figsize=[14,8])
for ii in range(4):
    key = schemes[ii]
    plt.plot(results[key]['N_coll_arr'], results[key][f'sum_dyn_err_2_array'], marker = 'o',label = titles[ii])
plt.grid()
plt.legend()
plt.yscale('log')
plt.title('Added second order dynamic error')
plt.xlabel('Number of collocation points')
plt.ylabel('Dynamic error ($rad/s^2$)')
plt.savefig(f'5_link_Sum_second_order_dynamic_error_vs_collocation_points_number.eps', format='eps')

In [ ]:
for scheme in schemes:
    key = scheme
    print('Problem:', key)
    N_arr = results[key]['N_arr']
    list_cpudt = []
    for N in N_arr:
        #print(f'\tN = {N}')
        list_cpudt.append(results[key][N]['cpudt'])
    results[key]['cpudt_array'] = np.array(list_cpudt)

In [ ]:
schemes = ['hs_mod_parab','hs_parab', 'trapz', 'trapz_mod']
titles = ['2nd order Hermite Simpson', 'Hermite Simpson','Trapezoidal', '2nd order Trapezoidal']
plt.figure(figsize=oct_fig_size)
for ii in [2,3,1,0]:
    key = schemes[ii]
    plt.plot(results[key]['N_arr'], results[key][f'cpudt_array'], marker = 'o', c = f'C{ii}', label = titles[ii])
plt.grid()
plt.legend()
#plt.yscale('log')
plt.title('5-link biped optimization time')
plt.xlabel('Number of intervals')
plt.ylabel('Time (s)')
plt.tight_layout(pad = 0.0)
plt.savefig(f'5_link_optimization_vs_interval_number.pdf', format='pdf')

schemes = ['hs_mod_parab','hs_parab', 'trapz', 'trapz_mod']
titles = ['Modified Hermite Simpson', 'Hermite Simpson','Trapezoidal', 'Modified Trapezoidal']
plt.figure(figsize=[14,8])
for ii in range(4):
    key = schemes[ii]
    plt.plot(results[key]['N_coll_arr'], results[key][f'cpudt_array'], marker = 'o',label = titles[ii])
plt.grid()
plt.legend()
#plt.yscale('log')
plt.title('optimization time')
plt.xlabel('Number of collocation points')
plt.ylabel('time (s)')
plt.savefig(f'5 link optimization vs collocation points number.eps', format='eps')

In [ ]:
for scheme in ['hs_mod_parab', 'hs_parab', 'trapz', 'trapz_mod']:
    key = scheme
    for N in [25,50]:#results[key]['N_arr']:
        print('scheme:', scheme, 'N:', N,'\n\ttime:', results[key][N][f'cpudt'],
              '\n\tErr 1:', results[key][N]['sum_dyn_err_q'], '\n\tErr 2:', results[key][N]['sum_dyn_err_2'])

## Animation

In [ ]:
from matplotlib import animation, rc
import matplotlib.patches as patches
from matplotlib.transforms import Affine2D
from IPython.display import HTML

In [ ]:
matplotlib.rcParams['animation.embed_limit'] = 200

In [ ]:
def body_tray(X, params):
    res = []
    for ii in range(X.shape[0]):
        res.append(list(chain_to_draw(X[ii,:], params)))
    return np.array(res)

In [ ]:
def loop_body_tray(X, params):
    point_tray = body_tray(X, params)
    point_tray_loop = np.append(
        point_tray,
        np.expand_dims(
            np.array(list(chain_to_draw(X[0,[4,3,2,1,0,5,6,7,8,9]],params)))
            ,0),
        0)
    return point_tray_loop

In [ ]:
def create_anim(X, U, params, n_loops = 1):
    [
    I_0_n, I_1_n, I_2_n, I_3_n, I_4_n,
    d_0_n, d_1_n, d_2_n, d_3_n, d_4_n,
    g_n,
    l_0_n, l_1_n, l_3_n,
    m_0_n, m_1_n, m_2_n, m_3_n, m_4_n
    ] = params
    
    N = X.shape[0]
    fig, ax = plt.subplots()
    draw_width = 14
    draw_height = 14
    
    fig.set_dpi(300)
    fig.set_size_inches([draw_width,draw_height])
    ax.set_xlim(( -1, 1))
    ax.set_ylim(( -0.2, 1.8))
    
    body, = ax.plot([], [], lw=4, ms = 12, marker = 'o')
    trail, = ax.plot([], [], lw=1, color = 'k')
    old_trail, = ax.plot([], [], lw=1, color = 'k')
    next_trail, = ax.plot([], [], lw=1, color = 'k')
    
    
    point_tray = body_tray(X, params)
    point_tray_loop = loop_body_tray(X, params)
    #sys_cm_point, = ax.plot([], [], 'go', ms=12)
    #line_sys_cm, = ax.plot([], [], 'k:', lw=1)
    
    print_vars = [X[:,ii] for ii in range(5)]+[np.linspace(0, N-1, N, dtype=int)]
    print_var_names = [f'q_{ii}' for ii in range(5)]+['step']
    texts = []
    ii = 0.8
    for arr in print_vars:
        texts.append(ax.text(-0.8, ii, "", fontsize = 12))
        ii -= 0.2
    
    ax.grid()
    
    def init():
        body.set_data([], [])
        trail.set_data(point_tray_loop[0,0,-1], point_tray_loop[0,1,-1])
        old_trail.set_data(point_tray_loop[:,0,-1]-0.5, point_tray_loop[:,1,-1])
        #next_trail.set_data(point_tray_loop[:,0,-1]+0.5, point_tray_loop[:,1,-1])
        #sys_cm_point.set_data([], [])
        #line_sys_cm.set_data([], [])
        return (body,)
    
    def animate(i):
        margin_x = -0.25 + i * 0.5/N
        trail.set_data(point_tray_loop[0:i+1,0,-1], point_tray_loop[0:i+1,1,-1])
        #sys_cm_coords = sys_cm_np(X[i,:], params)
        #sys_cm_point.set_data(sys_cm_coords)
        #line_sys_cm.set_data([0, sys_cm_coords[0]], [0, sys_cm_coords[1]])
        
        ax.set_xlim(( -1+ margin_x, 1+ margin_x))
        points_x, points_y = point_tray[i,:,:]
        body.set_data(points_x, points_y) 
        
        for ii in range(len(texts)):
            text = texts[ii]
            name = print_var_names[ii]
            arr = print_vars[ii]
            text.set_position((-0.9 + margin_x, 1.7 - 0.05*ii))
            if name == 'step':
                text.set_text("$step$ = " + str(arr[i]))
            else:
                text.set_text("$" + name + "$ = %.3f" % arr[i])
        return (body,)
    iterable_frames = sum([[jj for jj in range(N)]for kk in range(n_loops)], start = [])
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=iterable_frames, interval=20, 
                               blit=True)
    return anim





In [ ]:
anim = create_anim(results['hs_mod_parab'][25]['x'][:-1,:],results['hs_mod_parab'][25]['u'], params, 20)

In [ ]:
HTML(anim.to_jshtml())

In [ ]:
f = r"biped_animation_hd.mp4" 
writervideo = animation.FFMpegWriter(fps=25//0.7) 
anim.save(f, writer=writervideo)

In [ ]:
def __create_anim_simp(X, U, params):
    [
    I_0_n, I_1_n, I_2_n, I_3_n, I_4_n,
    d_0_n, d_1_n, d_2_n, d_3_n, d_4_n,
    g_n,
    l_0_n, l_1_n, l_3_n,
    m_0_n, m_1_n, m_2_n, m_3_n, m_4_n
    ] = params
    
    N = X.shape[0]
    fig, ax = plt.subplots()
    
    y_scale = 1
    min_x_cart = np.min(X[:,0])
    max_x_cart = np.max(X[:,0])
    cart_displ = max_x_cart-min_x_cart
    size_x = 2*y_scale + cart_displ
    size_y = 2*y_scale
    draw_width = 14
    draw_height = draw_width / size_x * size_y
    
    x_0 = X[:,0]
    y_0 = np.zeros_like(x_0)
    x_1 = x_0 + l_3*np.sin(X[:,1])
    y_1 = y_0 - l_3*np.cos(X[:,1])
    
    x_cm = (m_0 * x_0 + m_2 * x_1)/(m_0 + m_2)
    y_cm = (m_0 * y_0 + m_2 * y_1)/(m_0 + m_2)

    fig.set_size_inches([draw_width,draw_height])
    ax.set_xlim(( min_x_cart-y_scale, max_x_cart+y_scale))
    ax.set_ylim(( -y_scale, y_scale))

    circle1 = plt.Circle((0, 0), l_3, color='b', ls = ":", fill=False)
    ax.add_artist(circle1)
    ax.plot([min_x_cart - l_3, max_x_cart + l_3], [0,0], 'k', lw=1, ls = ':')

    line1, = ax.plot([], [], lw=2)
    line3, = ax.plot([], [], 'k', lw=1, ls = ':')
    line_cm, = ax.plot([], [], 'g', lw=1, ls = ':')
    point0, = ax.plot([], [], marker='o', markersize=15, color="red")
    point1, = ax.plot([], [], marker='o', markersize=15, color="red")
    point_cm, = ax.plot([], [], marker='o', markersize=10, color="green")
    u_max = max(np.max(np.abs(U[:,0])),1e-15)
    arrow_w = 0.05*l_3
    arrow_l = 0.2*l_3
    u_arrow = patches.Arrow(0, 0, 0, -arrow_l, color = 'gray',width = arrow_w)
    ax.add_patch(u_arrow)
    
    print_vars = [X[:,0], X[:,1], U[:,0], U[:,1], np.linspace(0, N-1, N, dtype=int)]
    print_var_names = ['q_0', 'q_1', 'u_0', 'u_1', 'step']
    texts = []
    ii = 0
    for arr in print_vars:
        texts.append(ax.text(0.2, ii, "", fontsize = 12))
        ii -= 0.08*l_3
    
    def init():
        line1.set_data([], [])
        line3.set_data([], [])
        line_cm.set_data([], [])
        point1.set_data([], [])
        circle1.center = (0, 0)
        return (line1,)
    def animate(i):
        circle1.center = (x_0[i], y_0[i])
        point0.set_data(x_0[i], y_0[i])
        line1.set_data([x_0[i], x_1[i]], [y_0[i], y_1[i]])    
        point1.set_data(x_1[i], y_1[i])
        point_cm.set_data(x_cm[i], y_cm[i])
        line3.set_data(x_1[:i], y_1[:i])
        line_cm.set_data(x_cm[:i], y_cm[:i])
        trans = Affine2D()
        u_arrow._patch_transform = trans.scale(U[i,0] * arrow_l / u_max, arrow_w).translate(x_0[i],0)
        for ii in range(len(texts)):
            text = texts[ii]
            name = print_var_names[ii]
            arr = print_vars[ii]
            text.set_text("$" + name + "$ = %.6f" % arr[i])
        return (line1,u_arrow)
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=N, interval=20, 
                               blit=True)
    return anim